In [18]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import re 
import plotly
from plotly import tools
from plotly import subplots
from plotly.graph_objs import Figure, Bar, Box, Histogram, Layout
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import folium                    
import folium.plugins as plugins 
from folium.plugins import HeatMapWithTime
%matplotlib inline

In [19]:
df2018 = pd.read_csv('df2018_1min.csv')
df2017 = pd.read_csv('df2017_1min.csv')
df2016 = pd.read_csv('df2016_1min.csv')
df2015 = pd.read_csv('df2015_1min.csv')
df2014 = pd.read_csv('df2014_1min.csv')
df2013 = pd.read_csv('df2013_1min.csv')

In [20]:
#Yearly crashes
crashes = [df2018, df2017, df2016, df2015, df2014, df2013]
years =['2018', '2017', '2016', '2015', '2014', '2013']
for i in range (len (years)):
    print('Number of crash in {}: {}'.format(years[i], crashes[i].shape[0]))

Number of crash in 2018: 1440
Number of crash in 2017: 1205
Number of crash in 2016: 859
Number of crash in 2015: 727
Number of crash in 2014: 772
Number of crash in 2013: 912


In [21]:
pd.set_option('display.max_columns',300)

In [22]:
df = pd.concat([df2013, df2014, df2015, df2016, df2017, df2018]).reset_index()

In [23]:
df.drop(columns = ['index', 'Unnamed: 0'], inplace = True)

In [24]:
#Remove 'conf' and 'cvalue' columns
conf_cols = [col for col in df.columns if 'conf' in col]
cvalue_cols = [col for col in df.columns if 'cvalue' in col]
c_cols = conf_cols + cvalue_cols

df.drop(columns = c_cols, inplace = True)

In [25]:
df.filter(regex='^pre_[0-9]+min').columns

Index(['pre_1min', 'pre_2min', 'pre_3min', 'pre_4min', 'pre_5min', 'pre_6min',
       'pre_7min', 'pre_8min', 'pre_9min', 'pre_10min', 'pre_11min',
       'pre_12min', 'pre_13min', 'pre_14min', 'pre_15min'],
      dtype='object')

In [26]:
#convert string to datetime
times1 = ['timestamp']
times2 = [t for t in df.filter(regex='^pre_[0-9]+min').columns]
times = times1 + times2

for time in times:
    df[time] = pd.to_datetime(df[time])

In [27]:
df.drop(columns=['DATE', 'TIME','ROUTE', 'RD_SYS'], inplace = True)

In [28]:
for col in df.columns:
    miss_col = df[col].isna().sum(axis=0)
    if miss_col > 0:
        print('{:<20s}{:>4d}'.format(col, miss_col))

CRASH_TYPE             1
LOC_DIR              395
RD_SURF                4
SURF_COND              9
LIGHT                  6
WEATHER                7
RD_DIV                18
RD_WRKZN              67
spd                  125
dn1_spd              454
dn2_spd              467
up1_spd              216
up2_spd              208
pre1_spd             109
pre1_dn1_spd         444
pre1_dn2_spd         458
pre1_up1_spd         204
pre1_up2_spd         197
pre2_spd             118
pre2_dn1_spd         453
pre2_dn2_spd         465
pre2_up1_spd         210
pre2_up2_spd         202
pre3_spd             127
pre3_dn1_spd         463
pre3_dn2_spd         476
pre3_up1_spd         218
pre3_up2_spd         208
pre4_spd             135
pre4_dn1_spd         469
pre4_dn2_spd         481
pre4_up1_spd         231
pre4_up2_spd         221
pre5_spd             134
pre5_dn1_spd         464
pre5_dn2_spd         478
pre5_up1_spd         226
pre5_up2_spd         217
pre6_spd             146
pre6_dn1_spd         479


It seems many INRIX data points are missing. Let's remove them from the dataframe

In [29]:
df.isna().sum(axis=0)
df.dropna(inplace=True)

In [30]:
df.shape

(4438, 131)

In [31]:
#speed difference among crash, downstream, and upstream TMCs
df['dn1_diff'] = df['spd'] - df['dn1_spd']
df['dn2_diff'] = df['spd'] - df['dn2_spd']
#df['dn3_diff'] = df['spd'] - df['dn3_spd']
df['up1_diff'] = df['spd'] - df['up1_spd']
df['up2_diff'] = df['spd'] - df['up2_spd']
#df['up3_diff'] = df['spd'] - df['up3_spd']
df['dn1dn2_diff'] = df['dn1_spd'] - df['dn2_spd']
#df['dn2dn3_diff'] = df['dn2_spd'] - df['dn3_spd']
df['up1up2_diff'] = df['up1_spd'] - df['up2_spd']
#df['up2up3_diff'] = df['up2_spd'] - df['up3_spd']

df['up1dn1_diff'] = df['up1_spd'] - df['dn1_spd']
df['up1dn2_diff'] = df['up1_spd'] - df['dn2_spd']
#df['up1dn3_diff'] = df['up1_spd'] - df['dn3_spd']

df['up2dn1_diff'] = df['up1_spd'] - df['dn1_spd']
df['up2dn2_diff'] = df['up1_spd'] - df['dn2_spd']
#df['up2dn3_diff'] = df['up1_spd'] - df['dn3_spd']

df['up3dn1_diff'] = df['up1_spd'] - df['dn1_spd']
df['up3dn2_diff'] = df['up1_spd'] - df['dn2_spd']
#df['up3dn3_diff'] = df['up1_spd'] - df['dn3_spd']

Let's develop a heatmap that exhibits the frequency of crasch along the route.

In [32]:
#Replace MP 10.9 to 10.8
df.replace(10.9, 10.8, inplace = True)

In [33]:
df['MP_ROUND'] =  df['MP'].round(1)

In [34]:
mp = pd.read_csv('NJ3_MP.csv')
mp.drop(columns = ['OBJECTID_1', 'Shape', 'OBJECTID', 'SRI', 'ROUTE', 'MUNICIPAL', 'COUNTY', 'SHAPE_LENG'], inplace = True)

In [35]:
df = pd.merge(df, mp, how = 'left', left_on = 'MP_ROUND', right_on = 'MILEPOST')

In [36]:
df['count'] =1

In [37]:
df18 = df[df['timestamp'].dt.year == 2018]
df17 = df[df['timestamp'].dt.year == 2017]
df16 = df[df['timestamp'].dt.year == 2016]
df15 = df[df['timestamp'].dt.year == 2015]
df14 = df[df['timestamp'].dt.year == 2014]
df13 = df[df['timestamp'].dt.year == 2013]

In [38]:
# Number of crash with processed dataset

crashes = [df18, df17, df16, df15, df14, df13]
years =['2018', '2017', '2016', '2015', '2014', '2013']
for i in range (len (years)):
    print('Number of crash in {}: {}'.format(years[i], crashes[i].shape[0]))

Number of crash in 2018: 1153
Number of crash in 2017: 991
Number of crash in 2016: 571
Number of crash in 2015: 530
Number of crash in 2014: 572
Number of crash in 2013: 690


In [39]:
df['timestamp']

0      2013-07-17 08:24:00
1      2013-05-02 10:04:00
2      2013-01-01 09:52:00
3      2013-01-19 15:16:00
4      2013-01-21 13:29:00
               ...        
4502   2018-12-23 21:36:00
4503   2018-12-26 17:49:00
4504   2018-12-27 08:58:00
4505   2018-12-28 13:36:00
4506   2018-12-28 17:07:00
Name: timestamp, Length: 4507, dtype: datetime64[ns]

#### 2018 Crashes

In [40]:
from folium.plugins import HeatMap

m = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 

HeatMap(data=df[['LATITUDE', 'LONGITUDE', 'count']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), 
        radius=10, max_zoom=10).add_to(m)

# folium.Marker([40.87111807,-74.18970011], popup='MP 0.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.85929634,-74.1803696 ], popup='MP 1.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.847009  ,-74.17049731], popup='MP 2.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.83791442,-74.1559026 ], popup='MP 3.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.83033654,-74.13963422], popup='MP 4.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.8226554 ,-74.12343017], popup='MP 5.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.81400146,-74.10816228], popup='MP 6.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.81104083,-74.0903372 ], popup='MP 7.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.80486343,-74.0738098 ], popup='MP 8.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.79512587,-74.05982774], popup='MP 9.0', icon=folium.Icon(color='red')).add_to(m)
# folium.Marker([40.78422417,-74.04775921], popup='MP 10.0', icon=folium.Icon(color='red')).add_to(m)

# m.save('heatmap2018.html')

m

#tiles = "Stamen Terrain"
# color_options = {'beige', 'black', 'blue', 'cadetblue', 'darkblue', 'darkgreen', 'darkpurple', 'darkred', 'gray', 
#'green', 'lightblue', 'lightgray', 'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'red', 'white'

#### 2013 Crashes

In [25]:
m = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 

HeatMap(data=df13[['LATITUDE', 'LONGITUDE', 'count']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), 
        radius=10, max_zoom=10).add_to(m)

folium.Marker([40.87111807,-74.18970011], popup='MP 0.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.85929634,-74.1803696 ], popup='MP 1.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.847009  ,-74.17049731], popup='MP 2.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.83791442,-74.1559026 ], popup='MP 3.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.83033654,-74.13963422], popup='MP 4.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.8226554 ,-74.12343017], popup='MP 5.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.81400146,-74.10816228], popup='MP 6.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.81104083,-74.0903372 ], popup='MP 7.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.80486343,-74.0738098 ], popup='MP 8.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.79512587,-74.05982774], popup='MP 9.0', icon=folium.Icon(color='red')).add_to(m)
folium.Marker([40.78422417,-74.04775921], popup='MP 10.0', icon=folium.Icon(color='red')).add_to(m)

m.save('heatmap2017.html')

m


In [49]:
m = folium.Map([40.820695, -74.119965], tiles = "stamentoner", zoom_start=12) 

HeatMap(data=df18[['LATITUDE', 'LONGITUDE', 'count']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), 
        radius=11, max_zoom=10).add_to(m)

# folium.CircleMarker([40.871484  ,-74.189490  ], radius =1, popup='MP 0.0', color='black').add_to(m)
# folium.CircleMarker([40.859394, -74.179865   ], radius =1, popup='MP 1.0', color='black').add_to(m)
# folium.CircleMarker([40.847009  ,-74.17049731], radius =3, popup='MP 2.0', color='black').add_to(m)
# folium.CircleMarker([40.83791442,-74.1559026 ], radius =3, popup='MP 3.0', color='black').add_to(m)
# folium.CircleMarker([40.83033654,-74.13963422], radius =3, popup='MP 4.0', color='black').add_to(m)
# folium.CircleMarker([40.8226554 ,-74.12343017], radius =3, popup='MP 5.0', color='black').add_to(m)
# folium.CircleMarker([40.81400146,-74.10816228], radius =3, popup='MP 6.0', color='black').add_to(m)
# folium.CircleMarker([40.81104083,-74.0903372 ], radius =3, popup='MP 7.0', color='black').add_to(m)
# folium.CircleMarker([40.80486343,-74.0738098 ], radius =3, popup='MP 8.0', color='black').add_to(m)
# folium.CircleMarker([40.79512587,-74.05982774], radius =3, popup='MP 9.0', color='black').add_to(m)
# folium.CircleMarker([40.78422417,-74.04775921], radius =3, popup='MP 10.0', color='black').add_to(m)

m

In [16]:
m = folium.Map([40.820695, -74.119965], tiles = "CartoDB Dark_Matter", zoom_start=12) 

HeatMap(data=df[['LATITUDE', 'LONGITUDE', 'count']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), 
        radius=11, max_zoom=10).add_to(m)

# folium.CircleMarker([40.871484  ,-74.189490  ], radius =3, popup='MP 0.0', color='black').add_to(m)
# folium.CircleMarker([40.859394, -74.179865   ], radius =3, popup='MP 1.0', color='black').add_to(m)
# folium.CircleMarker([40.847009  ,-74.17049731], radius =3, popup='MP 2.0', color='black').add_to(m)
# folium.CircleMarker([40.83791442,-74.1559026 ], radius =3, popup='MP 3.0', color='black').add_to(m)
# folium.CircleMarker([40.83033654,-74.13963422], radius =3, popup='MP 4.0', color='black').add_to(m)
# folium.CircleMarker([40.8226554 ,-74.12343017], radius =3, popup='MP 5.0', color='black').add_to(m)
# folium.CircleMarker([40.81400146,-74.10816228], radius =3, popup='MP 6.0', color='black').add_to(m)
# folium.CircleMarker([40.81104083,-74.0903372 ], radius =3, popup='MP 7.0', color='black').add_to(m)
# folium.CircleMarker([40.80486343,-74.0738098 ], radius =3, popup='MP 8.0', color='black').add_to(m)
# folium.CircleMarker([40.79512587,-74.05982774], radius =3, popup='MP 9.0', color='black').add_to(m)
# folium.CircleMarker([40.78422417,-74.04775921], radius =3, popup='MP 10.0', color='black').add_to(m)

m

KeyError: "None of [Index(['LATITUDE', 'LONGITUDE', 'count'], dtype='object')] are in the [columns]"

## Crash Locations for the period of 2013 - 2018

In [44]:
d1 = ['2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', 
     '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', 
     '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', 
     '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', 
     '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', 
     '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', 
     '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', 
     '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', 
     '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', 
     '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
     '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', 
     '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

d2 = list(range(1, 73))

In [45]:
data = {'date':d1, 'Weight':d2}
crash = pd.DataFrame.from_dict(data)

In [46]:
heat_df = df
heat_df['year_month'] = heat_df['timestamp'].dt.to_period('M').astype(str)
heat_df = pd.merge(heat_df, crash, how='left', left_on ='year_month', right_on ='date')
heat_df = heat_df[['LATITUDE', 'LONGITUDE', 'Weight']]

In [47]:
m_all = folium.Map([40.820695, -74.119965], tiles = "stamentoner", zoom_start=12) 
heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df[heat_df['Weight'] == i].iterrows()] for i in range(0,73)]

hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(m_all)
m_all

## Crash Locations by Year and Month

In [32]:
#if the date format comes in datetime, we can also extract the day/month/year using the to_period function
#where 'D', 'M', 'Y' are inputs
# df['month_year'] = pd.to_datetime(df['timestamp']).dt.to_period('M')
# df['month_year'].head()

In [33]:
# Montly crashes: 2013 - 2018
heat_df18 = df18
heat_df18['Weight'] = heat_df18['timestamp'].dt.month.astype(float)
heat_df18 = heat_df18.loc[:, ('LATITUDE', 'LONGITUDE', 'Weight')]

heat_df17 = df17
heat_df17['Weight'] = heat_df17['timestamp'].dt.month.astype(float)
heat_df17 = heat_df17.loc[:, ['LATITUDE', 'LONGITUDE', 'Weight']]

heat_df16 = df16
heat_df16['Weight'] = heat_df16['timestamp'].dt.month.astype(float)
heat_df16 = heat_df16.loc[:, ['LATITUDE', 'LONGITUDE', 'Weight']]

heat_df15 = df15
heat_df15['Weight'] = heat_df15['timestamp'].dt.month.astype(float)
heat_df15 = heat_df15.loc[:, ['LATITUDE', 'LONGITUDE', 'Weight']]

heat_df14 = df14
heat_df14['Weight'] = heat_df14['timestamp'].dt.month.astype(float)
heat_df14 = heat_df14.loc[:, ['LATITUDE', 'LONGITUDE', 'Weight']]

heat_df13 = df13
heat_df13['Weight'] = heat_df13['timestamp'].dt.month.astype(float)
heat_df13 = heat_df13.loc[:, ['LATITUDE', 'LONGITUDE', 'Weight']]

C:\Users\kkim1\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kkim1\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\kkim1\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

### 2018 Crashes by Month

In [34]:
# crash_heat = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in time_df18[time_df18['weight'] == i].iterrows()] for i in range(0, 13)]
m = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 
heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df18[heat_df18['Weight'] == i].iterrows()] for i in range(0,13)]

# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(m)
# Display the map
m

### 2017 Crashes by Month

In [35]:
m1 = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 
heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df17[heat_df17['Weight'] == i].iterrows()] for i in range(0,13)]

hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(m1)
m1

### 2016 Crashes by Month

In [36]:
m2 = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 
heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df16[heat_df16['Weight'] == i].iterrows()] for i in range(0,13)]

hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(m2)
m2

### 2015 Crashes by Month

In [37]:
m3 = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 
heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df15[heat_df15['Weight'] == i].iterrows()] for i in range(0,13)]

hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(m3)
m3

### 2014 Crashes by Month

In [38]:
m4 = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 
heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df14[heat_df14['Weight'] == i].iterrows()] for i in range(0,13)]

hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)

hm.add_to(m4)

m4.save('heatmap2014_time.html')
m4

### 2013 Crashes by Month

In [39]:
m5 = folium.Map([40.820695, -74.119965], tiles = "Stamen Terrain", zoom_start=12) 
heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df13[heat_df13['Weight'] == i].iterrows()] for i in range(0,13)]

hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(m5)
m5

## Crash frequency over time and location

In [40]:
df_3d = df
df_3d['Crash Month'] = df_3d['timestamp'].dt.month
df_3d['Crash Hour'] = df_3d['timestamp'].dt.hour
df_3d['Year'] =df_3d['timestamp'].dt.year
df_3d = df_3d[['MP', 'Crash Month', 'Crash Hour', 'spd', 'SEVERITY', 'Year', 'LOC_DIR']]

In [41]:
df_3d = df_3d.groupby(['Year','MP','Crash Month', 'Crash Hour', 'SEVERITY', 'LOC_DIR']).count().reset_index().values
df_3d

array([[2013, 0.48, 12, ..., 'I', 'E', 1],
       [2013, 0.51, 2, ..., 'I', 'E', 1],
       [2013, 0.51, 5, ..., 'I', 'E', 1],
       ...,
       [2018, 10.84, 12, ..., 'P', 'W', 2],
       [2018, 10.84, 12, ..., 'P', 'E', 2],
       [2018, 10.84, 12, ..., 'P', 'E', 2]], dtype=object)

In [42]:
df_3d = pd.DataFrame(df_3d,columns=['Year','MP', 'Crash Month', 'Crash Hour', 'SEVERITY', 'LOC_DIR', 'Count'])

In [43]:
df_3d.head()

,Year,MP,Crash Month,Crash Hour,SEVERITY,LOC_DIR,Count
0,2013,0.48,12,8,I,E,1
1,2013,0.51,2,18,I,E,1
2,2013,0.51,5,13,I,E,1
3,2013,0.51,7,9,P,E,1
4,2013,0.51,8,12,P,E,1


In [44]:
df_3d['Year'] = df_3d['Year'].astype(int)
df_3d['MP'] = df_3d['MP'].astype(float)
df_3d['Crash Month'] = df_3d['Crash Month'].astype(int)
df_3d['Crash Hour'] = df_3d['Crash Hour'].astype(int)
df_3d['Count'] = df_3d['Count'].astype(int)

In [ ]:
import plotly.express as px
data1 = df_3d.groupby(['Year', 'MP', 'LOC_DIR']).sum().reset_index()

fig = px.scatter_3d(data1, x='MP', y='Year', z='Count', size= 'Count', color = 'LOC_DIR', size_max=40,  
                    color_continuous_scale = 'Viridis', opacity=0.7)

# tight layout
fig.update_layout(title = 'Yearly Crash Frequency over Distance', autosize = True,
                  width = 1000, height = 800)

In [ ]:
data1 = df_3d.groupby(['Year', 'Crash Month', 'LOC_DIR']).sum().reset_index()

fig = px.scatter_3d(data1, x='Crash Month', y='Year', z='Count', size= 'Count', color = 'LOC_DIR', size_max=40,  
                    color_continuous_scale = 'Viridis', opacity=0.7)

# tight layout
fig.update_layout(title = 'Crash Frequency by Year and Month', autosize = True,
                  width = 1000, height = 800)

In [ ]:
data1 = df_3d.groupby(['Year', 'Crash Hour', 'LOC_DIR']).sum().reset_index()

fig = px.scatter_3d(data1, x='Crash Hour', y='Year', z='Count', size= 'Count', color = 'LOC_DIR', size_max=40,  
                    color_continuous_scale = 'Viridis', opacity=0.7)

# tight layout
fig.update_layout(title = 'Hourly Crash Frequency by Year', autosize = True,
                  width = 1000, height = 800)

## ASTI Volume Data Integration

In [ ]:
import datetime
from datetime import timedelta 

def round_to_5min(t):
    delta = datetime.timedelta(minutes=t.minute%5, 
                               seconds=t.second)
    t -= delta
    if delta > datetime.timedelta(0):
        t += datetime.timedelta(minutes=5)
    return t

In [ ]:
df['TIME_ROUND'] = df['timestamp'].map(round_to_5min)

In [ ]:
df.head()

In [ ]:
#Load ASTI date

qt11 = pd.read_csv('qt11_.csv')
qt12 = pd.read_csv('qt12_.csv')
qt13 = pd.read_csv('qt13_.csv')
qt14 = pd.read_csv('qt14_.csv')
qt15 = pd.read_csv('qt15_.csv')
qt2 = pd.read_csv('qt2_.csv')
qt3 = pd.read_csv('qt3_.csv')
qt4 = pd.read_csv('qt4_.csv')
qt6 = pd.read_csv('qt6_.csv')
qt7 = pd.read_csv('qt7_.csv')
qt8 = pd.read_csv('qt8_.csv')
qt9 = pd.read_csv('qt9_.csv')
asti = pd.read_csv('ASTI_5_MinVolume2018.csv')

In [ ]:
qt = [qt11, qt12, qt13, qt14, qt15, qt2, qt3, qt4, qt6, qt7, qt8, qt9]

for i in qt:
    i.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
for i in qt:
    i['MIN_INTVL_05'] = pd.to_datetime(i['MIN_INTVL_05'])

In [ ]:
n = []
for i in qt:
    n.append(i['ASTI_NAME'].iloc[0])

n

In [ ]:
qt11 = qt11.set_index('MIN_INTVL_05').resample('5T').mean()
qt12 = qt12.set_index('MIN_INTVL_05').resample('5T').mean()
qt13 = qt13.set_index('MIN_INTVL_05').resample('5T').mean()
qt14 = qt14.set_index('MIN_INTVL_05').resample('5T').mean()
qt15 = qt15.set_index('MIN_INTVL_05').resample('5T').mean()
qt2 = qt2.set_index('MIN_INTVL_05').resample('5T').mean()
qt3 = qt3.set_index('MIN_INTVL_05').resample('5T').mean()
qt4 = qt4.set_index('MIN_INTVL_05').resample('5T').mean()
qt6 = qt6.set_index('MIN_INTVL_05').resample('5T').mean()
qt7 = qt7.set_index('MIN_INTVL_05').resample('5T').mean()
qt8 = qt8.set_index('MIN_INTVL_05').resample('5T').mean()
qt9 = qt9.set_index('MIN_INTVL_05').resample('5T').mean()

In [ ]:
# qt11['ASTI_NAME'] = 'QT11'
# qt12['ASTI_NAME'] = 'QT12'
# qt13['ASTI_NAME'] = 'QT13'
# qt14['ASTI_NAME'] = 'QT14'
# qt15['ASTI_NAME'] = 'QT15'
# qt2['ASTI_NAME'] = 'QT2'
# #qt3['ASTI_NAME'] = 'QT3'
# qt4['ASTI_NAME'] = 'QT4'
# qt6['ASTI_NAME'] = 'QT6'
# qt7['ASTI_NAME'] = 'QT7'
# qt8['ASTI_NAME'] = 'QT8'
# qt9['ASTI_NAME'] = 'QT9'

qt11['ASTI_NAME'] = 'W_MP_8.0'
qt12['ASTI_NAME'] = 'E_MP_4.0'
qt13['ASTI_NAME'] = 'W_MP_7.0'
qt14['ASTI_NAME'] = 'E_MP_2.1'
qt15['ASTI_NAME'] = 'E_MP_2.9'
qt2['ASTI_NAME'] = 'E_MP_3.4'
#qt3['ASTI_NAME'] = 'QT3'
qt4['ASTI_NAME'] = 'W_MP_6.0'
qt6['ASTI_NAME'] = 'W_MP_8.5'
qt7['ASTI_NAME'] = 'E_MP_4.8'
qt8['ASTI_NAME'] = 'W_MP_7.5'
qt9['ASTI_NAME'] = 'W_MP_6.5'

# QT11 Route 3 WB MM 8
# QT12 Route 3 EB MM 4
# QT13 Route 3 WB MM 7
# QT14 Route 3 EB MM 2.1
# QT15 Route 3 EB MM 2.9
# QT2 Route 3 EB MM 3.4
# QT3 or QT10Route 17 S MM 3.9
# QT4 Route 3 WB MM 6
# QT6 Route 3 WB MM 8.5
# QT7 Route 3 EB MM 4.8
# QT8 Route 3 WB MM 7.5
# QT9 Route 3 WB MM 6.5

In [ ]:
qt11.reset_index(inplace=True)
qt12.reset_index(inplace=True)
qt13.reset_index(inplace=True)
qt14.reset_index(inplace=True)
qt15.reset_index(inplace=True)
qt2.reset_index(inplace=True)
qt3.reset_index(inplace=True)
qt4.reset_index(inplace=True)
qt6.reset_index(inplace=True)
qt7.reset_index(inplace=True)
qt8.reset_index(inplace=True)
qt9.reset_index(inplace=True)


In [ ]:
asti.drop(columns = 'LANE_GROUP', inplace = True)

In [ ]:
asti = asti[['MIN_INTVL_05', 'VOLUME', 'ASTI_NAME']]
asti

In [ ]:
asti.drop([457831], inplace = True)

In [ ]:
asti

In [ ]:
asti = pd.concat([asti, qt11, qt12, qt13, qt14, qt15, qt2, qt3, qt4, qt6, qt7, qt8, qt9], axis = 0)

In [ ]:
asti['ASTI_NAME'].value_counts()

## Speed Mean, Standard Deviation and Coefficient of Variation

In [ ]:
#Mean Speed for the previous 15 minutes

df['pre15_mean_spd']     = df[['pre1_spd','pre2_spd','pre3_spd', 'pre4_spd', 'pre5_spd','pre6_spd','pre7_spd','pre8_spd','pre9_spd','pre10_spd', 
                           'pre11_spd', 'pre12_spd', 'pre13_spd', 'pre14_spd', 'pre15_spd']].mean(axis=1)
df['pre15_dn1_mean_spd'] = df[['pre1_dn1_spd','pre2_dn1_spd','pre3_dn1_spd', 'pre4_dn1_spd','pre5_dn1_spd','pre6_dn1_spd','pre7_dn1_spd','pre8_dn1_spd',
                               'pre9_dn1_spd','pre10_dn1_spd','pre11_dn1_spd','pre12_dn1_spd','pre13_dn1_spd','pre14_dn1_spd','pre15_dn1_spd']].mean(axis=1)
df['pre15_dn2_mean_spd'] = df[['pre1_dn2_spd','pre2_dn2_spd','pre3_dn2_spd', 'pre4_dn2_spd','pre5_dn2_spd','pre6_dn2_spd','pre7_dn2_spd','pre8_dn2_spd',
                               'pre9_dn2_spd','pre10_dn2_spd','pre11_dn2_spd','pre12_dn2_spd','pre13_dn2_spd','pre14_dn2_spd','pre15_dn2_spd']].mean(axis=1)
df['pre15_dn3_mean_spd'] = df[['pre1_dn3_spd','pre2_dn3_spd','pre3_dn3_spd', 'pre4_dn3_spd','pre5_dn3_spd','pre6_dn3_spd','pre7_dn3_spd','pre8_dn3_spd',
                               'pre9_dn3_spd','pre10_dn3_spd','pre11_dn3_spd','pre12_dn3_spd','pre13_dn3_spd','pre14_dn3_spd','pre15_dn3_spd']].mean(axis=1)
df['pre15_up1_mean_spd'] = df[['pre1_up1_spd','pre2_up1_spd','pre3_up1_spd', 'pre4_up1_spd','pre5_up1_spd','pre6_up1_spd','pre7_up1_spd','pre8_up1_spd',
                               'pre9_up1_spd','pre10_up1_spd','pre11_up1_spd','pre12_up1_spd','pre13_up1_spd','pre14_up1_spd','pre15_up1_spd']].mean(axis=1)
df['pre15_up2_mean_spd'] = df[['pre1_up2_spd','pre2_up2_spd','pre3_up2_spd', 'pre4_up2_spd','pre5_up2_spd','pre6_up2_spd','pre7_up2_spd','pre8_up2_spd',
                               'pre9_up2_spd','pre10_up2_spd','pre11_up2_spd','pre12_up2_spd','pre13_up2_spd','pre14_up2_spd','pre15_up2_spd']].mean(axis=1)
df['pre15_up3_mean_spd'] = df[['pre1_up3_spd','pre2_up3_spd','pre3_up3_spd', 'pre4_up3_spd','pre5_up3_spd','pre6_up3_spd','pre7_up3_spd','pre8_up3_spd',
                               'pre9_up3_spd','pre10_up3_spd','pre11_up3_spd','pre12_up3_spd','pre13_up3_spd','pre14_up3_spd','pre15_up3_spd']].mean(axis=1)

In [ ]:
#Mean Speed for the previous 10 - 15 minutes
df['pre10_15_mean_spd']     = df[['pre11_spd', 'pre12_spd', 'pre13_spd', 'pre14_spd', 'pre15_spd']].mean(axis=1)
df['pre10_15_dn1_mean_spd'] = df[['pre11_dn1_spd','pre12_dn1_spd','pre13_dn1_spd','pre14_dn1_spd','pre15_dn1_spd']].mean(axis=1)
df['pre10_15_dn2_mean_spd'] = df[['pre11_dn2_spd','pre12_dn2_spd','pre13_dn2_spd','pre14_dn2_spd','pre15_dn2_spd']].mean(axis=1)
df['pre10_15_dn3_mean_spd'] = df[['pre11_dn3_spd','pre12_dn3_spd','pre13_dn3_spd','pre14_dn3_spd','pre15_dn3_spd']].mean(axis=1)
df['pre10_15_up1_mean_spd'] = df[['pre11_up1_spd','pre12_up1_spd','pre13_up1_spd','pre14_up1_spd','pre15_up1_spd']].mean(axis=1)
df['pre10_15_up2_mean_spd'] = df[['pre11_up2_spd','pre12_up2_spd','pre13_up2_spd','pre14_up2_spd','pre15_up2_spd']].mean(axis=1)
df['pre10_15_up3_mean_spd'] = df[['pre11_up3_spd','pre12_up3_spd','pre13_up3_spd','pre14_up3_spd','pre15_up3_spd']].mean(axis=1)

In [ ]:
#Mean Speed for the previous 10 minutes

df['pre10_mean_spd']     = df[['pre1_spd','pre2_spd','pre3_spd', 'pre4_spd', 'pre5_spd','pre6_spd','pre7_spd','pre8_spd','pre9_spd','pre10_spd']].mean(axis=1)
df['pre10_dn1_mean_spd'] = df[['pre1_dn1_spd','pre2_dn1_spd','pre3_dn1_spd', 'pre4_dn1_spd','pre5_dn1_spd','pre6_dn1_spd','pre7_dn1_spd','pre8_dn1_spd',
                               'pre9_dn1_spd','pre10_dn1_spd']].mean(axis=1)
df['pre10_dn2_mean_spd'] = df[['pre1_dn2_spd','pre2_dn2_spd','pre3_dn2_spd', 'pre4_dn2_spd','pre5_dn2_spd','pre6_dn2_spd','pre7_dn2_spd','pre8_dn2_spd',
                               'pre9_dn2_spd','pre10_dn2_spd']].mean(axis=1)
df['pre10_dn3_mean_spd'] = df[['pre1_dn3_spd','pre2_dn3_spd','pre3_dn3_spd', 'pre4_dn3_spd','pre5_dn3_spd','pre6_dn3_spd','pre7_dn3_spd','pre8_dn3_spd',
                               'pre9_dn3_spd','pre10_dn3_spd']].mean(axis=1)
df['pre10_up1_mean_spd'] = df[['pre1_up1_spd','pre2_up1_spd','pre3_up1_spd', 'pre4_up1_spd','pre5_up1_spd','pre6_up1_spd','pre7_up1_spd','pre8_up1_spd',
                               'pre9_up1_spd','pre10_up1_spd']].mean(axis=1)
df['pre10_up2_mean_spd'] = df[['pre1_up2_spd','pre2_up2_spd','pre3_up2_spd', 'pre4_up2_spd','pre5_up2_spd','pre6_up2_spd','pre7_up2_spd','pre8_up2_spd',
                               'pre9_up2_spd','pre10_up2_spd']].mean(axis=1)
df['pre10_up3_mean_spd'] = df[['pre1_up3_spd','pre2_up3_spd','pre3_up3_spd', 'pre4_up3_spd','pre5_up3_spd','pre6_up3_spd','pre7_up3_spd','pre8_up3_spd',
                               'pre9_up3_spd','pre10_up3_spd']].mean(axis=1)

In [ ]:
#Mean Speed for the previous 5 - 10 minutes

df['pre5_10_mean_spd']     = df[['pre6_spd','pre7_spd','pre8_spd','pre9_spd','pre10_spd']].mean(axis=1)
df['pre5_10_dn1_mean_spd'] = df[['pre6_dn1_spd','pre7_dn1_spd','pre8_dn1_spd','pre9_dn1_spd','pre10_dn1_spd']].mean(axis=1)
df['pre5_10_dn2_mean_spd'] = df[['pre6_dn2_spd','pre7_dn2_spd','pre8_dn2_spd','pre9_dn2_spd','pre10_dn2_spd']].mean(axis=1)
df['pre5_10_dn3_mean_spd'] = df[['pre6_dn3_spd','pre7_dn3_spd','pre8_dn3_spd','pre9_dn3_spd','pre10_dn3_spd']].mean(axis=1)
df['pre5_10_up1_mean_spd'] = df[['pre6_up1_spd','pre7_up1_spd','pre8_up1_spd','pre9_up1_spd','pre10_up1_spd']].mean(axis=1)
df['pre5_10_up2_mean_spd'] = df[['pre6_up2_spd','pre7_up2_spd','pre8_up2_spd','pre9_up2_spd','pre10_up2_spd']].mean(axis=1)
df['pre5_10_up3_mean_spd'] = df[['pre6_up3_spd','pre7_up3_spd','pre8_up3_spd','pre9_up3_spd','pre10_up3_spd']].mean(axis=1)

In [ ]:
#Mean Speed for the previous 5 minutes
df['pre5_mean_spd']     = df[['pre1_spd','pre2_spd','pre3_spd', 'pre4_spd', 'pre5_spd']].mean(axis=1)
df['pre5_dn1_mean_spd'] = df[['pre1_dn1_spd','pre2_dn1_spd','pre3_dn1_spd', 'pre4_dn1_spd','pre5_dn1_spd']].mean(axis=1)
df['pre5_dn2_mean_spd'] = df[['pre1_dn2_spd','pre2_dn2_spd','pre3_dn2_spd', 'pre4_dn2_spd','pre5_dn2_spd']].mean(axis=1)
df['pre5_dn3_mean_spd'] = df[['pre1_dn3_spd','pre2_dn3_spd','pre3_dn3_spd', 'pre4_dn3_spd','pre5_dn3_spd']].mean(axis=1)
df['pre5_up1_mean_spd'] = df[['pre1_up1_spd','pre2_up1_spd','pre3_up1_spd', 'pre4_up1_spd','pre5_up1_spd']].mean(axis=1)
df['pre5_up2_mean_spd'] = df[['pre1_up2_spd','pre2_up2_spd','pre3_up2_spd', 'pre4_up2_spd','pre5_up2_spd']].mean(axis=1)
df['pre5_up3_mean_spd'] = df[['pre1_up3_spd','pre2_up3_spd','pre3_up3_spd', 'pre4_up3_spd','pre5_up3_spd']].mean(axis=1)

In [ ]:
#Standard Deviation for the previous 15 minutes
df['pre15_std_spd']     = df[['pre1_spd','pre2_spd','pre3_spd', 'pre4_spd', 'pre5_spd','pre6_spd','pre7_spd','pre8_spd','pre9_spd','pre10_spd', 
                           'pre11_spd', 'pre12_spd', 'pre13_spd', 'pre14_spd', 'pre15_spd']].std(axis=1)
df['pre15_dn1_std_spd'] = df[['pre1_dn1_spd','pre2_dn1_spd','pre3_dn1_spd', 'pre4_dn1_spd','pre5_dn1_spd','pre6_dn1_spd','pre7_dn1_spd','pre8_dn1_spd',
                               'pre9_dn1_spd','pre10_dn1_spd','pre11_dn1_spd','pre12_dn1_spd','pre13_dn1_spd','pre14_dn1_spd','pre15_dn1_spd']].std(axis=1)
df['pre15_dn2_std_spd'] = df[['pre1_dn2_spd','pre2_dn2_spd','pre3_dn2_spd', 'pre4_dn2_spd','pre5_dn2_spd','pre6_dn2_spd','pre7_dn2_spd','pre8_dn2_spd',
                               'pre9_dn2_spd','pre10_dn2_spd','pre11_dn2_spd','pre12_dn2_spd','pre13_dn2_spd','pre14_dn2_spd','pre15_dn2_spd']].std(axis=1)
df['pre15_dn3_std_spd'] = df[['pre1_dn3_spd','pre2_dn3_spd','pre3_dn3_spd', 'pre4_dn3_spd','pre5_dn3_spd','pre6_dn3_spd','pre7_dn3_spd','pre8_dn3_spd',
                               'pre9_dn3_spd','pre10_dn3_spd','pre11_dn3_spd','pre12_dn3_spd','pre13_dn3_spd','pre14_dn3_spd','pre15_dn3_spd']].std(axis=1)
df['pre15_up1_std_spd'] = df[['pre1_up1_spd','pre2_up1_spd','pre3_up1_spd', 'pre4_up1_spd','pre5_up1_spd','pre6_up1_spd','pre7_up1_spd','pre8_up1_spd',
                               'pre9_up1_spd','pre10_up1_spd','pre11_up1_spd','pre12_up1_spd','pre13_up1_spd','pre14_up1_spd','pre15_up1_spd']].std(axis=1)
df['pre15_up2_std_spd'] = df[['pre1_up2_spd','pre2_up2_spd','pre3_up2_spd', 'pre4_up2_spd','pre5_up2_spd','pre6_up2_spd','pre7_up2_spd','pre8_up2_spd',
                               'pre9_up2_spd','pre10_up2_spd','pre11_up2_spd','pre12_up2_spd','pre13_up2_spd','pre14_up2_spd','pre15_up2_spd']].std(axis=1)
df['pre15_up3_std_spd'] = df[['pre1_up3_spd','pre2_up3_spd','pre3_up3_spd', 'pre4_up3_spd','pre5_up3_spd','pre6_up3_spd','pre7_up3_spd','pre8_up3_spd',
                               'pre9_up3_spd','pre10_up3_spd','pre11_up3_spd','pre12_up3_spd','pre13_up3_spd','pre14_up3_spd','pre15_up3_spd']].std(axis=1)

In [ ]:
#Standard Deviation Speed for the previous 10 - 15 minutes
df['pre10_15_std_spd']     = df[['pre11_spd', 'pre12_spd', 'pre13_spd', 'pre14_spd', 'pre15_spd']].std(axis=1)
df['pre10_15_dn1_std_spd'] = df[['pre11_dn1_spd','pre12_dn1_spd','pre13_dn1_spd','pre14_dn1_spd','pre15_dn1_spd']].std(axis=1)
df['pre10_15_dn2_std_spd'] = df[['pre11_dn2_spd','pre12_dn2_spd','pre13_dn2_spd','pre14_dn2_spd','pre15_dn2_spd']].std(axis=1)
df['pre10_15_dn3_std_spd'] = df[['pre11_dn3_spd','pre12_dn3_spd','pre13_dn3_spd','pre14_dn3_spd','pre15_dn3_spd']].std(axis=1)
df['pre10_15_up1_std_spd'] = df[['pre11_up1_spd','pre12_up1_spd','pre13_up1_spd','pre14_up1_spd','pre15_up1_spd']].std(axis=1)
df['pre10_15_up2_std_spd'] = df[['pre11_up2_spd','pre12_up2_spd','pre13_up2_spd','pre14_up2_spd','pre15_up2_spd']].std(axis=1)
df['pre10_15_up3_std_spd'] = df[['pre11_up3_spd','pre12_up3_spd','pre13_up3_spd','pre14_up3_spd','pre15_up3_spd']].std(axis=1)

In [ ]:
#Standard Deviation Speed for the previous 10 minutes
df['pre10_std_spd']     = df[['pre1_spd','pre2_spd','pre3_spd', 'pre4_spd', 'pre5_spd','pre6_spd','pre7_spd','pre8_spd','pre9_spd','pre10_spd']].std(axis=1)
df['pre10_dn1_std_spd'] = df[['pre1_dn1_spd','pre2_dn1_spd','pre3_dn1_spd', 'pre4_dn1_spd','pre5_dn1_spd','pre6_dn1_spd','pre7_dn1_spd','pre8_dn1_spd',
                               'pre9_dn1_spd','pre10_dn1_spd']].std(axis=1)
df['pre10_dn2_std_spd'] = df[['pre1_dn2_spd','pre2_dn2_spd','pre3_dn2_spd', 'pre4_dn2_spd','pre5_dn2_spd','pre6_dn2_spd','pre7_dn2_spd','pre8_dn2_spd',
                               'pre9_dn2_spd','pre10_dn2_spd']].std(axis=1)
df['pre10_dn3_std_spd'] = df[['pre1_dn3_spd','pre2_dn3_spd','pre3_dn3_spd', 'pre4_dn3_spd','pre5_dn3_spd','pre6_dn3_spd','pre7_dn3_spd','pre8_dn3_spd',
                               'pre9_dn3_spd','pre10_dn3_spd']].std(axis=1)
df['pre10_up1_std_spd'] = df[['pre1_up1_spd','pre2_up1_spd','pre3_up1_spd', 'pre4_up1_spd','pre5_up1_spd','pre6_up1_spd','pre7_up1_spd','pre8_up1_spd',
                               'pre9_up1_spd','pre10_up1_spd']].std(axis=1)
df['pre10_up2_std_spd'] = df[['pre1_up2_spd','pre2_up2_spd','pre3_up2_spd', 'pre4_up2_spd','pre5_up2_spd','pre6_up2_spd','pre7_up2_spd','pre8_up2_spd',
                               'pre9_up2_spd','pre10_up2_spd']].std(axis=1)
df['pre10_up3_std_spd'] = df[['pre1_up3_spd','pre2_up3_spd','pre3_up3_spd', 'pre4_up3_spd','pre5_up3_spd','pre6_up3_spd','pre7_up3_spd','pre8_up3_spd',
                               'pre9_up3_spd','pre10_up3_spd']].std(axis=1)

In [ ]:
#std Speed for the previous 5 - 10 minutes

df['pre5_10_std_spd']     = df[['pre6_spd','pre7_spd','pre8_spd','pre9_spd','pre10_spd']].std(axis=1)
df['pre5_10_dn1_std_spd'] = df[['pre6_dn1_spd','pre7_dn1_spd','pre8_dn1_spd','pre9_dn1_spd','pre10_dn1_spd']].std(axis=1)
df['pre5_10_dn2_std_spd'] = df[['pre6_dn2_spd','pre7_dn2_spd','pre8_dn2_spd','pre9_dn2_spd','pre10_dn2_spd']].std(axis=1)
df['pre5_10_dn3_std_spd'] = df[['pre6_dn3_spd','pre7_dn3_spd','pre8_dn3_spd','pre9_dn3_spd','pre10_dn3_spd']].std(axis=1)
df['pre5_10_up1_std_spd'] = df[['pre6_up1_spd','pre7_up1_spd','pre8_up1_spd','pre9_up1_spd','pre10_up1_spd']].std(axis=1)
df['pre5_10_up2_std_spd'] = df[['pre6_up2_spd','pre7_up2_spd','pre8_up2_spd','pre9_up2_spd','pre10_up2_spd']].std(axis=1)
df['pre5_10_up3_std_spd'] = df[['pre6_up3_spd','pre7_up3_spd','pre8_up3_spd','pre9_up3_spd','pre10_up3_spd']].std(axis=1)

In [ ]:
#std Speed for the previous 5 minutes

df['pre5_std_spd']     = df[['pre1_spd','pre2_spd','pre3_spd', 'pre4_spd', 'pre5_spd']].std(axis=1)
df['pre5_dn1_std_spd'] = df[['pre1_dn1_spd','pre2_dn1_spd','pre3_dn1_spd', 'pre4_dn1_spd','pre5_dn1_spd']].std(axis=1)
df['pre5_dn2_std_spd'] = df[['pre1_dn2_spd','pre2_dn2_spd','pre3_dn2_spd', 'pre4_dn2_spd','pre5_dn2_spd']].std(axis=1)
df['pre5_dn3_std_spd'] = df[['pre1_dn3_spd','pre2_dn3_spd','pre3_dn3_spd', 'pre4_dn3_spd','pre5_dn3_spd']].std(axis=1)
df['pre5_up1_std_spd'] = df[['pre1_up1_spd','pre2_up1_spd','pre3_up1_spd', 'pre4_up1_spd','pre5_up1_spd']].std(axis=1)
df['pre5_up2_std_spd'] = df[['pre1_up2_spd','pre2_up2_spd','pre3_up2_spd', 'pre4_up2_spd','pre5_up2_spd']].std(axis=1)
df['pre5_up3_std_spd'] = df[['pre1_up3_spd','pre2_up3_spd','pre3_up3_spd', 'pre4_up3_spd','pre5_up3_spd']].std(axis=1)

In [ ]:
#Coefficient of Variation

df['pre15_cv'] = df['pre15_std_spd']/df['pre15_mean_spd'] 
df['pre15_dn1_cv'] = df['pre15_dn1_std_spd']/df['pre15_dn1_mean_spd']
df['pre15_dn2_cv'] = df['pre15_dn2_std_spd']/df['pre15_dn2_mean_spd']
df['pre15_dn3_cv'] = df['pre15_dn3_std_spd']/df['pre15_dn3_mean_spd']
df['pre15_up1_cv'] = df['pre15_up1_std_spd']/df['pre15_up1_mean_spd']
df['pre15_up2_cv'] = df['pre15_up2_std_spd']/df['pre15_up2_mean_spd']
df['pre15_up3_cv'] = df['pre15_up3_std_spd']/df['pre15_up3_mean_spd']

In [ ]:
df['pre10_15_cv'] = df['pre10_15_std_spd']/df['pre10_15_mean_spd'] 
df['pre10_15_dn1_cv'] = df['pre10_15_dn1_std_spd']/df['pre10_15_dn1_mean_spd']
df['pre10_15_dn2_cv'] = df['pre10_15_dn2_std_spd']/df['pre10_15_dn2_mean_spd']
df['pre10_15_dn3_cv'] = df['pre10_15_dn3_std_spd']/df['pre10_15_dn3_mean_spd']
df['pre10_15_up1_cv'] = df['pre10_15_up1_std_spd']/df['pre10_15_up1_mean_spd']
df['pre10_15_up2_cv'] = df['pre10_15_up2_std_spd']/df['pre10_15_up2_mean_spd']
df['pre10_15_up3_cv'] = df['pre10_15_up3_std_spd']/df['pre10_15_up3_mean_spd']

In [ ]:
df['pre10_cv'] = df['pre10_std_spd']/df['pre10_mean_spd'] 
df['pre10_dn1_cv'] = df['pre10_dn1_std_spd']/df['pre10_dn1_mean_spd']
df['pre10_dn2_cv'] = df['pre10_dn2_std_spd']/df['pre10_dn2_mean_spd']
df['pre10_dn3_cv'] = df['pre10_dn3_std_spd']/df['pre10_dn3_mean_spd']
df['pre10_up1_cv'] = df['pre10_up1_std_spd']/df['pre10_up1_mean_spd']
df['pre10_up2_cv'] = df['pre10_up2_std_spd']/df['pre10_up2_mean_spd']
df['pre10_up3_cv'] = df['pre10_up3_std_spd']/df['pre10_up3_mean_spd']

In [ ]:
df['pre5_10_cv'] = df['pre5_10_std_spd']/df['pre5_10_mean_spd'] 
df['pre5_10_dn1_cv'] = df['pre5_10_dn1_std_spd']/df['pre5_10_dn1_mean_spd']
df['pre5_10_dn2_cv'] = df['pre5_10_dn2_std_spd']/df['pre5_10_dn2_mean_spd']
df['pre5_10_dn3_cv'] = df['pre5_10_dn3_std_spd']/df['pre5_10_dn3_mean_spd']
df['pre5_10_up1_cv'] = df['pre5_10_up1_std_spd']/df['pre5_10_up1_mean_spd']
df['pre5_10_up2_cv'] = df['pre5_10_up2_std_spd']/df['pre5_10_up2_mean_spd']
df['pre5_10_up3_cv'] = df['pre5_10_up3_std_spd']/df['pre5_10_up3_mean_spd']

In [ ]:
df['pre5_cv'] = df['pre5_std_spd']/df['pre5_mean_spd'] 
df['pre5_dn1_cv'] = df['pre5_dn1_std_spd']/df['pre5_dn1_mean_spd']
df['pre5_dn2_cv'] = df['pre5_dn2_std_spd']/df['pre5_dn2_mean_spd']
df['pre5_dn3_cv'] = df['pre5_dn3_std_spd']/df['pre5_dn3_mean_spd']
df['pre5_up1_cv'] = df['pre5_up1_std_spd']/df['pre5_up1_mean_spd']
df['pre5_up2_cv'] = df['pre5_up2_std_spd']/df['pre5_up2_mean_spd']
df['pre5_up3_cv'] = df['pre5_up3_std_spd']/df['pre5_up3_mean_spd']

In [ ]:
df.head()

In [ ]:
import seaborn as sns, numpy as np
# sns.set(); np.random.seed(0)
x = df['spd'].values
ax = sns.distplot(x)

In [ ]:
pre15_mean_spd = df[['pre15_mean_spd']]
pre15_mean_spd['Time'] = 'pre 15 min'
pre10_15_mean_spd = df[['pre10_15_mean_spd']]
pre10_15_mean_spd['Time'] = 'pre 10-15 min'
pre10_mean_spd = df[['pre10_mean_spd']]
pre10_mean_spd['Time'] = 'pre 10 min'
pre5_10_mean_spd = df[['pre5_10_mean_spd']]
pre5_10_mean_spd['Time'] = 'pre 5-10 min'
pre5_mean_spd  = df[['pre5_mean_spd']]
pre5_mean_spd['Time'] = 'pre 5 min'

In [ ]:
pre15_mean_spd.rename(columns = {'pre15_mean_spd':'Mean Speed', 'Time':'time'}, inplace = True)
pre10_15_mean_spd.rename(columns = {'pre10_15_mean_spd':'Mean Speed', 'Time':'time'}, inplace = True)
pre10_mean_spd.rename(columns = {'pre10_mean_spd':'Mean Speed', 'Time':'time'}, inplace = True)
pre5_10_mean_spd.rename(columns = {'pre5_10_mean_spd':'Mean Speed', 'Time':'time'}, inplace = True)
pre5_mean_spd.rename(columns = {'pre5_mean_spd':'Mean Speed', 'Time':'time'}, inplace = True)

In [ ]:
pre_mean_spd = pd.concat([pre5_mean_spd, pre5_10_mean_spd, pre10_mean_spd, pre10_15_mean_spd, pre15_mean_spd], axis = 0)

In [ ]:
pre_mean_spd

In [ ]:
ordered_mean_spd = pre_mean_spd.time.value_counts().index
g = sns.FacetGrid( pre_mean_spd, row="time", row_order=ordered_mean_spd,
                  height=1.7, aspect=4,)
g.map(sns.distplot, "Mean Speed", hist=False, rug=True);

In [ ]:
pre15_std_spd = df[['pre15_std_spd']]
pre15_std_spd['Time'] = 'pre 15 min'
pre10_std_spd = df[['pre10_std_spd']]
pre10_std_spd['Time'] = 'pre 10 min'
pre10_15_std_spd = df[['pre10_15_std_spd']]
pre10_15_std_spd['Time'] = 'pre 10-15 min'
pre5_10_std_spd = df[['pre5_10_std_spd']]
pre5_10_std_spd['Time'] = 'pre 5-10 min'
pre5_std_spd  = df[['pre5_std_spd']]
pre5_std_spd['Time'] = 'pre 5 min'

In [ ]:
pre15_std_spd.rename(columns = {'pre15_std_spd':'Speed std', 'Time':'time'}, inplace = True)
pre10_std_spd.rename(columns = {'pre10_std_spd':'Speed std', 'Time':'time'}, inplace = True)
pre10_15_std_spd.rename(columns = {'pre10_15_std_spd':'Speed std', 'Time':'time'}, inplace = True)
pre5_10_std_spd.rename(columns = {'pre5_10_std_spd':'Speed std', 'Time':'time'}, inplace = True)
pre5_std_spd.rename(columns = {'pre5_std_spd':'Speed std', 'Time':'time'}, inplace = True)

In [ ]:
pre_std_spd = pd.concat([pre5_std_spd, pre10_std_spd, pre15_std_spd, pre5_10_std_spd, pre10_15_std_spd], axis = 0)

In [ ]:
ordered_std_spd = pre_std_spd.time.value_counts().index
g = sns.FacetGrid( pre_std_spd, row="time", row_order=ordered_std_spd,
                  height=1.7, hue = 'time', aspect=4, )
g.map(sns.distplot, "Speed std", hist=False, rug=True);

In [ ]:
import seaborn as sns

# Sort the dataframe by target
target_0 = pre_std_spd[pre_std_spd['time'] == 'pre 15 min']
target_1 = pre_std_spd[pre_std_spd['time'] == 'pre 10 min']
target_2 = pre_std_spd[pre_std_spd['time'] == 'pre 5 min']

sns.distplot(target_0[['Speed std']], label='pre 15 min', hist=False, rug=True)
sns.distplot(target_1[['Speed std']], label='pre 10 min',hist=False, rug=True)
sns.distplot(target_2[['Speed std']], label='pre 5 min',hist=False, rug=True)

plt.xlabel('Speed STD')
plt.legend()
plt.show()


In [ ]:
pre15_std_spd	pre15_dn1_std_spd	pre15_dn2_std_spd	pre15_dn3_std_spd	pre15_up1_std_spd	pre15_up2_std_spd	pre15_up3_std_spd	

In [ ]:
pre15_cv = df[['pre15_cv']]
pre15_cv['Time'] = 'pre 15 min'
pre10_cv = df[['pre10_cv']]
pre10_cv['Time'] = 'pre 10 min'
pre10_15_cv = df[['pre10_15_cv']]
pre10_15_cv['Time'] = 'pre 10-15 min'
pre5_10_cv = df[['pre5_10_cv']]
pre5_10_cv['Time'] = 'pre 5-10 min'
pre5_cv  = df[['pre5_cv']]
pre5_cv['Time'] = 'pre 5 min'

In [ ]:
pre15_cv.rename(columns = {'pre15_cv':'cv', 'Time':'time'}, inplace = True)
pre10_cv.rename(columns = {'pre10_cv':'cv', 'Time':'time'}, inplace = True)
pre10_15_cv.rename(columns = {'pre10_15_cv':'cv', 'Time':'time'}, inplace = True)
pre5_10_cv.rename(columns = {'pre5_10_cv':'cv', 'Time':'time'}, inplace = True)
pre5_cv.rename(columns = {'pre5_cv':'cv', 'Time':'time'}, inplace = True)

In [ ]:
pre_cv1 = pd.concat([pre5_cv, pre10_cv, pre15_cv], axis = 0)
pre_cv2 = pd.concat([pre5_cv, pre5_10_cv, pre10_15_cv], axis = 0)

In [ ]:
ordered_cv = pre_cv2.time.value_counts().index
g = sns.FacetGrid(pre_cv2, row="time", row_order=ordered_cv,
                  height=1.7, hue = 'time', aspect=4, )
g.map(sns.distplot, "cv", hist=False, rug=True);

In [ ]:
ordered_cv = pre_cv1.time.value_counts().index
g = sns.FacetGrid( pre_cv1, row="time", row_order=ordered_cv,
                  height=1.7, hue = 'time', aspect=4, )
g.map(sns.distplot, "cv", hist=False, rug=True);

In [ ]:
tips = sns.load_dataset("tips")

In [ ]:
tips

In [ ]:
ordered_days = tips.day.value_counts().index
g = sns.FacetGrid(tips, row="day", row_order=ordered_days,
                  height=1.7, aspect=4,)
g.map(sns.distplot, "total_bill", hist=False, rug=True);

In [ ]:
import plotly.express as px
df2 = px.data.tips()
fig = px.histogram(df2, x="total_bill", y="tip", color="sex",
                   marginal="box", # or violin, rug
                   hover_data=df2.columns)
fig.show()

In [ ]:
df2

In [ ]:
df_3d.dtypes

In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Get Data: this ex will only use part of it (i.e. rows 750-1500)
df4 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

start, end = 750, 1500

fig = go.Figure(data=go.Scatter3d(
    x=df_3d['Year'][start:end],
    y=df_3d['MP'][start:end],
    z=df_3d['Count'][start:end],
    text=df_3d['Count'][start:end],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=750,
        size=df_3d['Count'][start:end],
        color = df_3d['Crash Month'][start:end],
        colorscale = 'Viridis',
        colorbar_title = 'Crash<br>Month',
        line_color='rgb(140, 140, 170)'
    )
))


fig.update_layout(height=800, width=800,
                  title='Examining Population and Life Expectancy Over Time')

fig.show()

In [ ]:
df3 = df

df3.rename(columns ={})
df3['Crash Month'] = df3['timestamp'].dt.month.astype(int)
df3['Crash Hour'] = df3['timestamp'].dt.hour.astype(int)
df3['Year'] =df3['timestamp'].dt.year.astype(int)
df3['Crash Spd'] = df3['spd'].astype(float)
df3['DN1 Spd'] = df3['dn1_spd'].astype(float)
df3['DN2 Spd'] = df3['dn2_spd'].astype(float)
df3['DN3 Spd'] = df3['dn3_spd'].astype(float)
df3['UP1 Spd'] = df3['up1_spd'].astype(float)
df3['UP2 Spd'] = df3['up2_spd'].astype(float)
df3['UP3 Spd'] = df3['up3_spd'].astype(float)
df3['TMC_LEN'] = df3['tmc_miles'].astype(float)
df3['TMCDN1_LEN'] = df3['tmc_dn1_miles'].astype(float)
df3['TMCDN2_LEN'] = df3['tmc_dn2_miles'].astype(float)
df3['TMCDN3_LEN'] = df3['tmc_dn3_miles'].astype(float)
df3['TMCUP1_LEN'] = df3['tmc_up1_miles'].astype(float)
df3['TMCUP2_LEN'] = df3['tmc_up2_miles'].astype(float)
df3['TMCUP3_LEN'] = df3['tmc_up3_miles'].astype(float)

df3 = df3[['Year','MP', 'Crash Month', 'Crash Hour', 'SEVERITY', 'LOC_DIR', 'Crash Spd', 'DN1 Spd',
           'DN2 Spd', 'DN3 Spd', 'UP1 Spd', 'UP2 Spd', 'UP3 Spd', 'TMC_LEN', 'TMCDN1_LEN','TMCDN2_LEN',
           'TMCDN3_LEN', 'TMCUP1_LEN', 'TMCUP2_LEN', 'TMCUP3_LEN']]

#'spd', 'dn1_spd', 'dn2_spd', 'dn3_spd', 'up1_spd', 'up2_spd', 'up3_spd'
#'tmc_miles', 'tmc_dn1_miles', 'tmc_dn2_miles', 'tmc_dn3_miles', 'tmc_up1_miles', 'tmc_up2_miles', 'tmc_up3_miles'

## Speed differences among consecutive TMCs

In [ ]:
df3.head()

In [ ]:
df4 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')
df4

In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Get Data: this ex will only use part of it (i.e. rows 750-1500)
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

start, end = 750, 1500

fig = go.Figure(data=go.Scatter3d(
    x=df['year'][start:end],
    y=df['continent'][start:end],
    z=df['pop'][start:end],
    text=df['country'][start:end],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=750,
        size=df['gdpPercap'][start:end],
        color = df['lifeExp'][start:end],
        colorscale = 'Viridis',
        colorbar_title = 'Life<br>Expectancy',
        line_color='rgb(140, 140, 170)'
    )
))


fig.update_layout(height=800, width=800,
                  title='Examining Population and Life Expectancy Over Time')

fig.show()

In [ ]:
df_3d.LOC_DIR.value_counts()

In [ ]:
df_3d = df
df_3d['crash_hour'] = df_3d['timestamp'].dt.time
df_3d['year'] =df_3d['timestamp'].dt.year
df_3d = df_3d[['MP', 'crash_hour', 'spd', 'year']]

In [ ]:
df_3d_2018 = df_3d[df_3d['year']==2018]
df_3d_2018.drop(columns=['year'], inplace = True)

In [ ]:
df_3d_2018 = df_3d_2018.groupby(['MP','crash_hour']).count().reset_index().values.tolist()

In [ ]:
df_3d_2018_ = pd.DataFrame(df_3d_2018,columns=['MP', 'Time', 'Count'])

In [ ]:
import plotly.express as px
df_3d = df_3d_2018_
fig = px.scatter_3d(df_3d, x='MP', y='Time', z='Count',
              color='Count', size='Count', size_max=18,
              opacity=0.7)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
ranges = [-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40]
fig, axes = plt.subplots(figsize=(10,4),nrows=1, ncols=2)
crash_dn1.groupby(pd.cut(crash_dn1, ranges)).count().plot(ax=axes[0], kind='bar')
crash_dn1w.groupby(pd.cut(crash_dn1w, ranges)).count().plot(ax=axes[1], kind='bar')
axes[0].set_xlabel('Spd diff crash TMC and 1st DN TMC')
axes[1].set_xlabel('Spd diff crash TMC and 1st DN TMC')
axes[0].set_title('EB')
axes[1].set_title('WB')

# Number of crashes per travel direction (EB - WB)

In [ ]:
df_2018 = df[df['timestamp'].dt.year == 2018]

crash_EB_mp = df[df['LOC_DIR']=='E']['MP']
crash_WB_mp = df[df['LOC_DIR']=='W']['MP']

trace_EB = go.Histogram(x=crash_EB_mp, name = 'EB Crash')
trace_WB = go.Histogram(x=crash_WB_mp, name = 'WB Crash')

fig = tools.make_subplots(rows=2, cols=1)
fig.append_trace(trace_EB, 1, 1)
fig.append_trace(trace_WB, 2, 1)
fig['layout'].update(height=800, width=800, title='Number of Crashes by MP', 
                     bargap = 0.3, xaxis1=dict(title="Mile Post (EB)"), yaxis1 = dict(title='Number of Crash'), 
                     xaxis2 = dict(title="Mile Post (WB)"), yaxis2 = dict(title='Number of Crash'), barmode='group')
iplot(fig)

# Number of Crashes by Time Period (EB - WB)

In [ ]:
crash_EB_tt = df[df['LOC_DIR']=='E']['timestamp'].dt.hour
crash_WB_tt = df[df['LOC_DIR']=='W']['timestamp'].dt.hour

trace_EB = go.Histogram(x=crash_EB_tt, name = 'EB Crash')
trace_WB = go.Histogram(x=crash_WB_tt, name = 'WB Crash')

fig = tools.make_subplots(rows=2, cols=1)
fig.append_trace(trace_EB, 1, 1)
fig.append_trace(trace_WB, 2, 1)
fig['layout'].update(height=800, width=800, title='Number of Crashes by Time Period', 
                     bargap = 0.3, xaxis1=dict(title="Hour"), yaxis1 = dict(title='Number of Crash (EB)'), 
                     xaxis2 = dict(title="Hour"), yaxis2 = dict(title='Number of Crash (WB)'), barmode='group')
iplot(fig)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')

xpos = [1,2,3,4,5,6,7,8,9,10]
ypos = [2,3,4,5,1,6,2,1,7,2]
num_elements = len(xpos)
zpos = [0,0,0,0,0,0,0,0,0,0]
dx = np.ones(10)
dy = np.ones(10)
dz = [1,2,3,4,5,6,7,8,9,10]

ax1.bar3d(xpos, ypos, zpos, dx, dy, dz, color='#00ceaa')
plt.show()

In [ ]:
df1 = pd.DataFrame(np.random.normal(50, 5, size=(300, 4)), columns=list('ABCD'))

In [ ]:
df1

In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
df[df['LOC_DIR']=='E']['timestamp'].dt.year.value_counts()

In [ ]:
df[df['LOC_DIR']=='W']['timestamp'].dt.year.value_counts()

# Crash Speed Distribution

In [ ]:
import seaborn as sns, numpy as np
# sns.set(); np.random.seed(0)
x = df['spd'].values
ax = sns.distplot(x)

Crash speed distributions are ranged between 4 mph and 72 mph with two peaks
Let's divide speeds into two groups: congested vs. uncongested. To this end, up and downstream links will be reviewed.

In [ ]:
df['SPD_MEAN_DN'] = df[['dn1_spd', 'dn2_spd', 'dn3_spd']].mean(axis=1).round(1)
df['SPD_STD_DN']  = df[['dn1_spd', 'dn2_spd', 'dn3_spd']].std(axis=1).round(1)
df['SPD_MEAN_UP'] = df[['up1_spd', 'up2_spd', 'up3_spd']].mean(axis=1).round(1)
df['SPD_STD_UP']  = df[['up1_spd', 'up2_spd', 'up3_spd']].std(axis=1).round(1)
df['SPD_DIFF_DN'] = np.round(df['spd'] - df['SPD_MEAN'])
df['SPD_DIFF_UP'] = df['spd'] - df['SPD_MEAN_UP']
df['SPD_DIFF_DNUP'] = df['SPD_MEAN'] - df['SPD_MEAN_UP']

In [ ]:
df.drop(columns = ['SPD_MEAN', 'SPD_STD'], inplace = True)

In [ ]:
df['SPD_STD'].max()

In [ ]:
df['SPD_STD'].min()

In [ ]:
t = df['SPD_STD'].values
ax = sns.distplot(t)

In [ ]:
df[df['spd'] <= 35]

Let's review the directional speed changes between the TMC where crashes occurred and upstream or downstream TMCs. 
If crash affected speeds at TMC where crash occurred, the first three figures would have more negative speed differences 
as speeds at downstream TMCs are higher.  

# Speed Differences by TMC (Upstream and Downstream)

### 1. speed difference =  (crash TMC - first downstream TMC)

#### Eastbound

In [ ]:
#Eastbound Crashes. Note that speed differences were estimated from speed at crash TMC - speed at downstream (or upstream) TMC

crash_dn1 = df[df['LOC_DIR']=='E']['dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='E']['dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='E']['dn3_diff']
crash_up1 = df[df['LOC_DIR']=='E']['up1_diff']
crash_up2 = df[df['LOC_DIR']=='E']['up2_diff']
crash_up3 = df[df['LOC_DIR']=='E']['up3_diff']


trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff w/ dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff w/ dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff w/ dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff w/ up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff w/ up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff w/ up3')

fig = tools.make_subplots(rows=3, cols=2)
fig.append_trace(trace_dn1, 1, 1)
fig.append_trace(trace_dn2, 1, 2)
fig.append_trace(trace_dn3, 2, 1)
fig.append_trace(trace_up1, 2, 2)
fig.append_trace(trace_up2, 3, 1)
fig.append_trace(trace_up3, 3, 2)

fig['layout'].update(title='Speed differences among TMCs (EB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis5 = dict(title="Speed (mph)"),
                     xaxis6 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

#### Westbound

In [ ]:
#Westbound Crashes
crash_dn1w = df[df['LOC_DIR']=='W']['dn1_diff']
crash_dn2w = df[df['LOC_DIR']=='W']['dn2_diff']
crash_dn3w = df[df['LOC_DIR']=='W']['dn3_diff']
crash_up1w = df[df['LOC_DIR']=='W']['up1_diff']
crash_up2w = df[df['LOC_DIR']=='W']['up2_diff']
crash_up3w = df[df['LOC_DIR']=='W']['up3_diff']


trace_dn1w = go.Histogram(x=crash_dn1w, name = 'Spd diff w/ dn1')
trace_dn2w = go.Histogram(x=crash_dn2w, name = 'Spd diff w/ dn2')
trace_dn3w = go.Histogram(x=crash_dn3w, name = 'Spd diff w/ dn3')
trace_up1w = go.Histogram(x=crash_up1w, name = 'Spd diff w/ up1')
trace_up2w = go.Histogram(x=crash_up2w, name = 'Spd diff w/ up2')
trace_up3w = go.Histogram(x=crash_up3w, name = 'Spd diff w/ up3')

fig = tools.make_subplots(rows=3, cols=2)
fig.append_trace(trace_dn1w, 1, 1)
fig.append_trace(trace_dn2w, 1, 2)
fig.append_trace(trace_dn3w, 2, 1)
fig.append_trace(trace_up1w, 2, 2)
fig.append_trace(trace_up2w, 3, 1)
fig.append_trace(trace_up3w, 3, 2)

fig['layout'].update(title='Speed differences among TMCs (WB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis5 = dict(title="Speed (mph)"),
                     xaxis6 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

###  2. Speed distributions by Upstream TMC 

In [ ]:
#Speed distributions by TMC
spd_bins = np.linspace(0, 70, 15)
fig, axes = plt.subplots(figsize=(20,4),nrows=1, ncols=4)
df['spd'].groupby(pd.cut(df['up1_spd'], spd_bins)).count().plot(ax = axes[0], kind = 'bar')
df['up1_spd'].groupby(pd.cut(df['up1_spd'], spd_bins)).count().plot(ax = axes[1], kind = 'bar')
df['up2_spd'].groupby(pd.cut(df['up1_spd'], spd_bins)).count().plot(ax = axes[2], kind = 'bar')
df['up3_spd'].groupby(pd.cut(df['up1_spd'], spd_bins)).count().plot(ax = axes[3], kind = 'bar')

axes[0].set_xlabel('Spd at TMC where crash occurred')
axes[1].set_xlabel('Spd at 1st Up TMC')
axes[2].set_xlabel('Spd at 2nd Up TMC')
axes[3].set_xlabel('Spd at 3rd Up TMC')


### 3. Speed differences between crash TMC and first downstream TMC

In [ ]:
ranges = [-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40]
fig, axes = plt.subplots(figsize=(10,4),nrows=1, ncols=2)
crash_dn1.groupby(pd.cut(crash_dn1, ranges)).count().plot(ax=axes[0], kind='bar')
crash_dn1w.groupby(pd.cut(crash_dn1w, ranges)).count().plot(ax=axes[1], kind='bar')
axes[0].set_xlabel('Spd diff crash TMC and 1st DN TMC')
axes[1].set_xlabel('Spd diff crash TMC and 1st DN TMC')
axes[0].set_title('EB')
axes[1].set_title('WB')

### 4. Speed differences between first upstream and downstream TMC

In [ ]:
#speed at upstream TMC1 - speed at downstream TMC1. Speed would be negative if crash affects traffic speed. 
crash_up1dn1 = df[df['LOC_DIR']=='E']['up1dn1_diff']
crash_up1dn1w = df[df['LOC_DIR']=='W']['up1dn1_diff']

ranges = [-70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60]

fig, axes = plt.subplots(figsize=(10,4),nrows=1, ncols=2)
crash_up1dn1.groupby(pd.cut(crash_up1dn1, ranges)).count().plot(ax=axes[0], kind='bar')
crash_up1dn1w.groupby(pd.cut(crash_up1dn1w, ranges)).count().plot(ax=axes[1], kind='bar')
axes[0].set_xlabel('Spd diff 1st UP and DN TMC')
axes[1].set_xlabel('Spd diff 1st UP and DN TMC')
axes[0].set_title('EB')
axes[1].set_title('WB')

# TMC Speed Difference vs. Weather Condition

In [ ]:
df.WEATHER.value_counts()

1: Clear <br/>
2: Rain <br/>
3: Snow <br/>
4: Fog <br/>
5-99: Other

In [ ]:
df_w1 = df[df['WEATHER'] == 1]
df_w2 = df[df['WEATHER'] == 2]
df_w3 = df[df['WEATHER'] == 3]
df_w4 = df[df['WEATHER'] == 4]
df_w5_ = df[df['WEATHER'].isin([0,5,6,7,8,99])]

### 1: Clear

In [ ]:
ranges=[-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40]
clear1 = df_w1['dn1_diff'].groupby(pd.cut(df_w1['dn1_diff'], ranges)).count()
clear2 = df_w1['up1dn1_diff'].groupby(pd.cut(df_w1['up1dn1_diff'], ranges)).count()

In [ ]:
fig, axes = plt.subplots(figsize=(10,4),nrows=1, ncols=2)
clear1.plot(ax=axes[0], kind='bar')
clear2.plot(ax=axes[1], kind='bar')

### 2: Rain

In [ ]:
ranges=[-40, -30, -20, -10, 0, 10, 20, 30, 40]
rain1 = df_w2['dn1_diff'].groupby(pd.cut(df_w2['dn1_diff'], ranges)).count()
rain2 = df_w2['up1dn1_diff'].groupby(pd.cut(df_w2['up1dn1_diff'], ranges)).count()

In [ ]:
fig, axes = plt.subplots(figsize=(10,4),nrows=1, ncols=2)
rain1.plot(ax=axes[0], kind='bar')
rain2.plot(ax=axes[1], kind='bar')

### 3: Snow

In [ ]:
ranges=[-40, -30, -20, -10, 0, 10, 20, 30, 40]
snow1 = df_w3['dn1_diff'].groupby(pd.cut(df_w3['dn1_diff'], ranges)).count()
snow2 = df_w3['up1dn1_diff'].groupby(pd.cut(df_w3['up1dn1_diff'], ranges)).count()

In [ ]:
fig, axes = plt.subplots(figsize=(10,4),nrows=1, ncols=2)
snow1.plot(ax=axes[0], kind='bar')
snow2.plot(ax=axes[1], kind='bar')

### 5-99: Other

In [ ]:
ranges=[-30, -20, -10, 0, 10, 20, 30]
other1 = df_w5_['dn1_diff'].groupby(pd.cut(df_w5_['dn1_diff'], ranges)).count()
other2 = df_w5_['up1dn1_diff'].groupby(pd.cut(df_w5_['up1dn1_diff'], ranges)).count()

In [ ]:
fig, axes = plt.subplots(figsize=(10,4),nrows=1, ncols=2)
other1.plot(ax=axes[0], kind='bar')
other2.plot(ax=axes[1], kind='bar')

# TMC Speed Difference vs. Reported Crash Time (5 min, 10 min, 15 min)

In [ ]:
#speed difference over 5-minutes
df['spd_pre5_diff'] = df['spd'] - df['pre5_spd']
df['spd_pre5_dn1_diff'] = df['dn1_spd'] - df['pre5_dn1_spd']
df['spd_pre5_dn2_diff'] = df['dn2_spd'] - df['pre5_dn2_spd']
df['spd_pre5_dn3_diff'] = df['dn3_spd'] - df['pre5_dn3_spd']
df['spd_pre5_up1_diff'] = df['up1_spd'] - df['pre5_up1_spd']
df['spd_pre5_up2_diff'] = df['up2_spd'] - df['pre5_up2_spd']
df['spd_pre5_up3_diff'] = df['up3_spd'] - df['pre5_up3_spd']

In [ ]:
df['spd_pre10_diff'] = df['spd'] - df['pre10_spd']
df['spd_pre10_dn1_diff'] = df['dn1_spd'] - df['pre10_dn1_spd']
df['spd_pre10_dn2_diff'] = df['dn2_spd'] - df['pre10_dn2_spd']
df['spd_pre10_dn3_diff'] = df['dn3_spd'] - df['pre10_dn3_spd']
df['spd_pre10_up1_diff'] = df['up1_spd'] - df['pre10_up1_spd']
df['spd_pre10_up2_diff'] = df['up2_spd'] - df['pre10_up2_spd']
df['spd_pre10_up3_diff'] = df['up3_spd'] - df['pre10_up3_spd']

In [ ]:
df['spd_pre15_diff'] = df['spd'] - df['pre15_spd']
df['spd_pre15_dn1_diff'] = df['dn1_spd'] - df['pre15_dn1_spd']
df['spd_pre15_dn2_diff'] = df['dn2_spd'] - df['pre15_dn2_spd']
df['spd_pre15_dn3_diff'] = df['dn3_spd'] - df['pre15_dn3_spd']
df['spd_pre15_up1_diff'] = df['up1_spd'] - df['pre15_up1_spd']
df['spd_pre15_up2_diff'] = df['up2_spd'] - df['pre15_up2_spd']
df['spd_pre15_up3_diff'] = df['up3_spd'] - df['pre15_up3_spd']

## Speed Variation over 5-Minute

In [ ]:
#Eastbound Speed Variation over 5 minutes. 
#Note that speed differences were estimated from speed at crash TMC - speed at downstream (or upstream) TMC

crash_link = df[df['LOC_DIR']=='E']['spd_pre5_diff']
crash_dn1= df[df['LOC_DIR']=='E']['spd_pre5_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='E']['spd_pre5_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='E']['spd_pre5_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='E']['spd_pre5_up1_diff']
crash_up2 = df[df['LOC_DIR']=='E']['spd_pre5_up2_diff']
crash_up3 = df[df['LOC_DIR']=='E']['spd_pre5_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=4, cols=2)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn2, 2, 2)
fig.append_trace(trace_dn3, 3, 1)
fig.append_trace(trace_up1, 3, 2)
fig.append_trace(trace_up2, 4, 1)
fig.append_trace(trace_up3, 4, 2)

fig['layout'].update(title='Speed Variation over 5-minute (EB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis7 = dict(title="Speed (mph)"),
                     xaxis8 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

In [ ]:
#Westbound Speed Variation over 5 minutes. 
#Note that speed differences were estimated from speed at crash TMC - speed at downstream (or upstream) TMC

crash_link = df[df['LOC_DIR']=='W']['spd_pre5_diff']
crash_dn1= df[df['LOC_DIR']=='W']['spd_pre5_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='W']['spd_pre5_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='W']['spd_pre5_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='W']['spd_pre5_up1_diff']
crash_up2 = df[df['LOC_DIR']=='W']['spd_pre5_up2_diff']
crash_up3 = df[df['LOC_DIR']=='W']['spd_pre5_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=4, cols=2)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn2, 2, 2)
fig.append_trace(trace_dn3, 3, 1)
fig.append_trace(trace_up1, 3, 2)
fig.append_trace(trace_up2, 4, 1)
fig.append_trace(trace_up3, 4, 2)

fig['layout'].update(title='Speed Variation over 5-minute (WB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis7 = dict(title="Speed (mph)"),
                     xaxis8 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

## TMC Speed Variation over 10-Minute

### Eastbound

In [ ]:
#Eastbound Speed Variation over 10 minutes. 
#Note that speed differences were estimated from speed at crash TMC - speed at downstream (or upstream) TMC

crash_link = df[df['LOC_DIR']=='E']['spd_pre10_diff']
crash_dn1= df[df['LOC_DIR']=='E']['spd_pre10_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='E']['spd_pre10_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='E']['spd_pre10_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='E']['spd_pre10_up1_diff']
crash_up2 = df[df['LOC_DIR']=='E']['spd_pre10_up2_diff']
crash_up3 = df[df['LOC_DIR']=='E']['spd_pre10_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=4, cols=2)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn2, 2, 2)
fig.append_trace(trace_dn3, 3, 1)
fig.append_trace(trace_up1, 3, 2)
fig.append_trace(trace_up2, 4, 1)
fig.append_trace(trace_up3, 4, 2)

fig['layout'].update(title='Speed Variation over 10-minute (EB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis5 = dict(title="Speed (mph)"),
                     xaxis6 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

### Westbound

In [ ]:
#Westbound Speed Variation over 10 minutes. 
#Note that speed differences were estimated from speed at crash TMC - speed at downstream (or upstream) TMC

crash_link = df[df['LOC_DIR']=='W']['spd_pre10_diff']
crash_dn1= df[df['LOC_DIR']=='W']['spd_pre10_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='W']['spd_pre10_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='W']['spd_pre10_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='W']['spd_pre10_up1_diff']
crash_up2 = df[df['LOC_DIR']=='W']['spd_pre10_up2_diff']
crash_up3 = df[df['LOC_DIR']=='W']['spd_pre10_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=4, cols=2)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn2, 2, 2)
fig.append_trace(trace_dn3, 3, 1)
fig.append_trace(trace_up1, 3, 2)
fig.append_trace(trace_up2, 4, 1)
fig.append_trace(trace_up3, 4, 2)

fig['layout'].update(title='Speed Variation over 10-minute (WB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis7 = dict(title="Speed (mph)"),
                     xaxis8 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

## TMC Speed Variation over 15-Minute

### Eastbound

In [ ]:
crash_link = df[df['LOC_DIR']=='E']['spd_pre15_diff']
crash_dn1= df[df['LOC_DIR']=='E']['spd_pre15_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='E']['spd_pre15_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='E']['spd_pre15_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='E']['spd_pre15_up1_diff']
crash_up2 = df[df['LOC_DIR']=='E']['spd_pre15_up2_diff']
crash_up3 = df[df['LOC_DIR']=='E']['spd_pre15_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=4, cols=2)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn2, 2, 2)
fig.append_trace(trace_dn3, 3, 1)
fig.append_trace(trace_up1, 3, 2)
fig.append_trace(trace_up2, 4, 1)
fig.append_trace(trace_up3, 4, 2)

fig['layout'].update(title='Speed Variation over 15-minute (EB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis7 = dict(title="Speed (mph)"),
                     xaxis8 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

In [ ]:
#Westbound Speed Variation over 15 minutes. 
#Note that speed differences were estimated from speed at crash TMC - speed at downstream (or upstream) TMC

crash_link = df[df['LOC_DIR']=='W']['spd_pre15_diff']
crash_dn1= df[df['LOC_DIR']=='W']['spd_pre15_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='W']['spd_pre15_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='W']['spd_pre15_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='W']['spd_pre15_up1_diff']
crash_up2 = df[df['LOC_DIR']=='W']['spd_pre15_up2_diff']
crash_up3 = df[df['LOC_DIR']=='W']['spd_pre15_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=4, cols=2)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn2, 2, 2)
fig.append_trace(trace_dn3, 3, 1)
fig.append_trace(trace_up1, 3, 2)
fig.append_trace(trace_up2, 4, 1)
fig.append_trace(trace_up3, 4, 2)

fig['layout'].update(title='Speed Variation over 15-minute (WB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis7 = dict(title="Speed (mph)"),
                     xaxis8 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

## Overll

### Eastbound

In [ ]:
crash_link = df[df['LOC_DIR']=='E']['spd_pre5_diff']
crash_dn1= df[df['LOC_DIR']=='E']['spd_pre5_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='E']['spd_pre5_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='E']['spd_pre5_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='E']['spd_pre5_up1_diff']
crash_up2 = df[df['LOC_DIR']=='E']['spd_pre5_up2_diff']
crash_up3 = df[df['LOC_DIR']=='E']['spd_pre5_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

crash_link1 = df[df['LOC_DIR']=='E']['spd_pre10_diff']
crash_dn11= df[df['LOC_DIR']=='E']['spd_pre10_dn1_diff']
crash_dn21 = df[df['LOC_DIR']=='E']['spd_pre10_dn2_diff']
crash_dn31 = df[df['LOC_DIR']=='E']['spd_pre10_dn3_diff']
crash_up11 = df[df['LOC_DIR']=='E']['spd_pre10_up1_diff']
crash_up21 = df[df['LOC_DIR']=='E']['spd_pre10_up2_diff']
crash_up31 = df[df['LOC_DIR']=='E']['spd_pre10_up3_diff']

trace_link1 = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn11 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn21 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn31 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up11 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up21 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up31 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

crash_link2 = df[df['LOC_DIR']=='E']['spd_pre15_diff']
crash_dn12= df[df['LOC_DIR']=='E']['spd_pre15_dn1_diff']
crash_dn22 = df[df['LOC_DIR']=='E']['spd_pre15_dn2_diff']
crash_dn32 = df[df['LOC_DIR']=='E']['spd_pre15_dn3_diff']
crash_up12 = df[df['LOC_DIR']=='E']['spd_pre15_up1_diff']
crash_up22 = df[df['LOC_DIR']=='E']['spd_pre15_up2_diff']
crash_up32 = df[df['LOC_DIR']=='E']['spd_pre15_up3_diff']

trace_link2 = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn12 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn22 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn32 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up12 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up22 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up32 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=7, cols=3)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_link1, 1, 2)
fig.append_trace(trace_link2, 1, 3)

fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn11, 2, 2)
fig.append_trace(trace_dn12, 2, 3)

fig.append_trace(trace_dn2, 3, 1)
fig.append_trace(trace_dn21, 3, 2)
fig.append_trace(trace_dn22, 3, 3)

fig.append_trace(trace_dn3, 4, 1)
fig.append_trace(trace_dn31, 4, 2)
fig.append_trace(trace_dn32, 4, 3)

fig.append_trace(trace_up1, 5, 1)
fig.append_trace(trace_up11, 5, 2)
fig.append_trace(trace_up12, 5, 3)

fig.append_trace(trace_up2, 6, 1)
fig.append_trace(trace_up21, 6, 2)
fig.append_trace(trace_up22, 6, 3)

fig.append_trace(trace_up3, 7, 1)
fig.append_trace(trace_up31, 7, 2)
fig.append_trace(trace_up32, 7, 3)



fig['layout'].update(title='Speed Variation over 5, 10, and 15-minute (EB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     yaxis1 = dict(title='Crash'),
                     yaxis4 = dict(title='DN1'),
                     yaxis7 = dict(title='DN2'),
                     yaxis10 = dict(title='DN3'),
                     yaxis13 = dict(title='UP1'),
                     yaxis16 = dict(title='UP2'),
                     yaxis19 = dict(title='UP3'),                  
                     xaxis19 = dict(title="Speed diff (mph) over 5-minute"),
                     xaxis20 = dict(title="Speed diff (mph) over 10-minute"),
                     xaxis21 = dict(title="Speed diff (mph) over 15-minute"), barmode='group')
iplot(fig)

### Westbound

In [ ]:
crash_link = df[df['LOC_DIR']=='W']['spd_pre5_diff']
crash_dn1= df[df['LOC_DIR']=='W']['spd_pre5_dn1_diff']
crash_dn2 = df[df['LOC_DIR']=='W']['spd_pre5_dn2_diff']
crash_dn3 = df[df['LOC_DIR']=='W']['spd_pre5_dn3_diff']
crash_up1 = df[df['LOC_DIR']=='W']['spd_pre5_up1_diff']
crash_up2 = df[df['LOC_DIR']=='W']['spd_pre5_up2_diff']
crash_up3 = df[df['LOC_DIR']=='W']['spd_pre5_up3_diff']

trace_link = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn1 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn2 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn3 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up1 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up2 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up3 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

crash_link1 = df[df['LOC_DIR']=='W']['spd_pre10_diff']
crash_dn11= df[df['LOC_DIR']=='W']['spd_pre10_dn1_diff']
crash_dn21 = df[df['LOC_DIR']=='W']['spd_pre10_dn2_diff']
crash_dn31 = df[df['LOC_DIR']=='W']['spd_pre10_dn3_diff']
crash_up11 = df[df['LOC_DIR']=='W']['spd_pre10_up1_diff']
crash_up21 = df[df['LOC_DIR']=='W']['spd_pre10_up2_diff']
crash_up31 = df[df['LOC_DIR']=='W']['spd_pre10_up3_diff']

trace_link1 = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn11 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn21 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn31 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up11 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up21 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up31 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

crash_link2 = df[df['LOC_DIR']=='W']['spd_pre15_diff']
crash_dn12= df[df['LOC_DIR']=='W']['spd_pre15_dn1_diff']
crash_dn22 = df[df['LOC_DIR']=='W']['spd_pre15_dn2_diff']
crash_dn32 = df[df['LOC_DIR']=='W']['spd_pre15_dn3_diff']
crash_up12 = df[df['LOC_DIR']=='W']['spd_pre15_up1_diff']
crash_up22 = df[df['LOC_DIR']=='W']['spd_pre15_up2_diff']
crash_up32 = df[df['LOC_DIR']=='W']['spd_pre15_up3_diff']

trace_link2 = go.Histogram(x=crash_link, name = 'Spd diff')
trace_dn12 = go.Histogram(x=crash_dn1, name = 'Spd diff dn1')
trace_dn22 = go.Histogram(x=crash_dn2, name = 'Spd diff dn2')
trace_dn32 = go.Histogram(x=crash_dn3, name = 'Spd diff dn3')
trace_up12 = go.Histogram(x=crash_up1, name = 'Spd diff up1')
trace_up22 = go.Histogram(x=crash_up2, name = 'Spd diff up2')
trace_up32 = go.Histogram(x=crash_up3, name = 'Spd diff up3')

fig = tools.make_subplots(rows=7, cols=3)
fig.append_trace(trace_link, 1, 1)
fig.append_trace(trace_link1, 1, 2)
fig.append_trace(trace_link2, 1, 3)

fig.append_trace(trace_dn1, 2, 1)
fig.append_trace(trace_dn11, 2, 2)
fig.append_trace(trace_dn12, 2, 3)

fig.append_trace(trace_dn2, 3, 1)
fig.append_trace(trace_dn21, 3, 2)
fig.append_trace(trace_dn22, 3, 3)

fig.append_trace(trace_dn3, 4, 1)
fig.append_trace(trace_dn31, 4, 2)
fig.append_trace(trace_dn32, 4, 3)

fig.append_trace(trace_up1, 5, 1)
fig.append_trace(trace_up11, 5, 2)
fig.append_trace(trace_up12, 5, 3)

fig.append_trace(trace_up2, 6, 1)
fig.append_trace(trace_up21, 6, 2)
fig.append_trace(trace_up22, 6, 3)

fig.append_trace(trace_up3, 7, 1)
fig.append_trace(trace_up31, 7, 2)
fig.append_trace(trace_up32, 7, 3)



fig['layout'].update(title='Speed Variation over 5, 10, and 15-minute (WB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     yaxis1 = dict(title='Crash'),
                     yaxis4 = dict(title='DN1'),
                     yaxis7 = dict(title='DN2'),
                     yaxis10 = dict(title='DN3'),
                     yaxis13 = dict(title='UP1'),
                     yaxis16 = dict(title='UP2'),
                     yaxis19 = dict(title='UP3'),                  
                     xaxis19 = dict(title="Speed diff (mph) over 5-minute"),
                     xaxis20 = dict(title="Speed diff (mph) over 10-minute"),
                     xaxis21 = dict(title="Speed diff (mph) over 15-minute"), barmode='group')
iplot(fig)

### Build Sub_df

In [ ]:
df_sub = df[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]
df_sub = df_sub.sort_values(by ='timestamp')

In [ ]:
df2018_sub = df_sub[df_sub['timestamp'].dt.year == 2018]
df2017_sub = df_sub[df_sub['timestamp'].dt.year == 2017]
df2016_sub = df_sub[df_sub['timestamp'].dt.year == 2016]
df2015_sub = df_sub[df_sub['timestamp'].dt.year == 2015]
df2014_sub = df_sub[df_sub['timestamp'].dt.year == 2014]
df2013_sub = df_sub[df_sub['timestamp'].dt.year == 2013]

In [ ]:
#timestamp and spd columns
# df2018_sub = df2018[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]
# df2017_sub = df2017[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]
# df2016_sub = df2016[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]
# df2015_sub = df2015[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]
# df2014_sub = df2014[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]
# df2013_sub = df2013[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]

In [ ]:
df2018_sub = df2018_sub.sort_values(by='timestamp')
df2017_sub = df2017_sub.sort_values(by='timestamp')
df2016_sub = df2016_sub.sort_values(by='timestamp')
df2015_sub = df2015_sub.sort_values(by='timestamp')
df2014_sub = df2014_sub.sort_values(by='timestamp')
df2013_sub = df2013_sub.sort_values(by='timestamp')

In [ ]:
#pre timestamp and spd columns
df_pre5  = df[['pre_5min', 'pre5_dn3_spd', 'pre5_dn2_spd', 'pre5_dn1_spd', 'pre5_spd', \
               'pre5_up1_spd', 'pre5_up2_spd', 'pre5_up3_spd']]
df_pre10 = df[['pre_10min', 'pre10_dn3_spd', 'pre10_dn2_spd', 'pre10_dn1_spd', 'pre10_spd', \
               'pre10_up1_spd', 'pre10_up2_spd', 'pre10_up3_spd']]
df_pre15 = df[['pre_15min', 'pre15_dn3_spd', 'pre15_dn2_spd', 'pre15_dn1_spd', 'pre15_spd', \
               'pre15_up1_spd', 'pre15_up2_spd', 'pre15_up3_spd']]

In [ ]:
df_pre5.rename(columns ={'pre5_dn3_spd':'DN3', 'pre5_dn2_spd':'DN2', 'pre5_dn1_spd':'DN1',
                         'pre5_spd':'CRASH', 'pre5_up1_spd':'UP1', 'pre5_up2_spd':'UP2', \
                         'pre5_up3_spd':'UP3'}, inplace = True)

df_pre10.rename(columns ={'pre10_dn3_spd':'DN3', 'pre10_dn2_spd':'DN2', 'pre10_dn1_spd':'DN1',
                          'pre10_spd':'CRASH', 'pre10_up1_spd':'UP1', 'pre10_up2_spd':'UP2', \
                          'pre10_up3_spd':'UP3'}, inplace = True)

df_pre15.rename(columns ={'pre15_dn3_spd':'DN3', 'pre15_dn2_spd':'DN2', 'pre15_dn1_spd':'DN1',
                          'pre15_spd':'CRASH', 'pre15_up1_spd':'UP1', 'pre15_up2_spd':'UP2', \
                          'pre15_up3_spd':'UP3'}, inplace = True)

### Build sub df

In [ ]:
#timestamp and spd columns
df_sub = df[['timestamp', 'dn3_spd', 'dn2_spd', 'dn1_spd', 'spd', 'up1_spd', 'up2_spd', 'up3_spd']]

In [ ]:
#post timestamp and spd columns
df_post5  = df[['post_5min', 'post5_dn3_spd', 'post5_dn2_spd', 'post5_dn1_spd', 'post5_spd', \
                'post5_up1_spd', 'post5_up2_spd', 'post5_up3_spd']]
df_post10 = df[['post_10min', 'post10_dn3_spd', 'post10_dn2_spd', 'post10_dn1_spd', 'post10_spd', \
                'post10_up1_spd', 'post10_up2_spd', 'post10_up3_spd']]
df_post15 = df[['post_15min', 'post15_dn3_spd', 'post15_dn2_spd', 'post15_dn1_spd', 'post15_spd', \
                'post15_up1_spd', 'post15_up2_spd', 'post15_up3_spd']]

### Rename Columns

In [ ]:
df2018_sub.rename(columns ={'dn3_spd':'DN3', 'dn2_spd':'DN2', 'dn1_spd':'DN1',
                        'spd':'crash', 'up1_spd':'UP1', 'up2_spd':'UP2', 'up3_spd':'UP3'}, inplace = True)

In [ ]:
df_pre5_2018 = df_pre5[df_pre5['pre_5min'].dt.year == 2018]
df_pre10_2018 = df_pre10[df_pre10['pre_10min'].dt.year == 2018]
df_pre15_2018 = df_pre15[df_pre15['pre_15min'].dt.year == 2018]

In [ ]:
df2018_sub = df2018_sub.sort_values(by='timestamp')

In [ ]:
df2018_sub_Jan = df2018_sub[df2018_sub['timestamp'].dt.month == 1]
df2018_sub_Feb = df2018_sub[df2018_sub['timestamp'].dt.month == 2]
df2018_sub_Mar = df2018_sub[df2018_sub['timestamp'].dt.month == 3]
df2018_sub_Apr = df2018_sub[df2018_sub['timestamp'].dt.month == 4]
df2018_sub_May = df2018_sub[df2018_sub['timestamp'].dt.month == 5]
df2018_sub_Jun = df2018_sub[df2018_sub['timestamp'].dt.month == 6]
df2018_sub_Jul = df2018_sub[df2018_sub['timestamp'].dt.month == 7] 
df2018_sub_Aug = df2018_sub[df2018_sub['timestamp'].dt.month == 8]
df2018_sub_Sep = df2018_sub[df2018_sub['timestamp'].dt.month == 9]
df2018_sub_Oct = df2018_sub[df2018_sub['timestamp'].dt.month == 10]
df2018_sub_Nov = df2018_sub[df2018_sub['timestamp'].dt.month == 11]
df2018_sub_Dec = df2018_sub[df2018_sub['timestamp'].dt.month == 12]

In [ ]:
#Speed 5-minute before crash
df_pre5_2018_Jan = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 1]
df_pre5_2018_Feb = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 2]
df_pre5_2018_Mar = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 3]
df_pre5_2018_Apr = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 4]
df_pre5_2018_May = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 5]
df_pre5_2018_Jun = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 6]
df_pre5_2018_Jul = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 7] 
df_pre5_2018_Aug = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 8]
df_pre5_2018_Sep = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 9]
df_pre5_2018_Oct = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 10]
df_pre5_2018_Nov = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 11]
df_pre5_2018_Dec = df_pre5_2018[df_pre5_2018['pre_5min'].dt.month == 12]

In [ ]:
#Speed 10-minute before crash
df_pre10_2018_Jan = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 1]
df_pre10_2018_Feb = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 2]
df_pre10_2018_Mar = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 3]
df_pre10_2018_Apr = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 4]
df_pre10_2018_May = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 5]
df_pre10_2018_Jun = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 6]
df_pre10_2018_Jul = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 7] 
df_pre10_2018_Aug = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 8]
df_pre10_2018_Sep = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 9]
df_pre10_2018_Oct = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 10]
df_pre10_2018_Nov = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 11]
df_pre10_2018_Dec = df_pre10_2018[df_pre10_2018['pre_10min'].dt.month == 12]

In [ ]:
#Speed 15-minute before crash
df_pre15_2018_Jan = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 1]
df_pre15_2018_Feb = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 2]
df_pre15_2018_Mar = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 3]
df_pre15_2018_Apr = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 4]
df_pre15_2018_May = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 5]
df_pre15_2018_Jun = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 6]
df_pre15_2018_Jul = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 7] 
df_pre15_2018_Aug = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 8]
df_pre15_2018_Sep = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 9]
df_pre15_2018_Oct = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 10]
df_pre15_2018_Nov = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 11]
df_pre15_2018_Dec = df_pre15_2018[df_pre15_2018['pre_15min'].dt.month == 12]

In [ ]:
#2018 January & February Crash Data

z1 = df2018_sub_Jan.iloc[:,1:]
x1 = df2018_sub_Jan.iloc[:,0]
y1 = df2018_sub_Jan.columns[1:]

z2 = df2018_sub_Feb.iloc[:,1:]
x2 = df2018_sub_Feb.iloc[:,0]
y2 = df2018_sub_Feb.columns[1:]

trace1 = go.Heatmap(z=z1, x=y1, y=x1, colorscale='Viridis')
trace2 = go.Heatmap(z=z2, x=y2, y=x2, colorbar=dict(x=0.43), colorscale='Viridis')

fig = tools.make_subplots(rows=1, cols=2, horizontal_spacing = 0.17)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(title='Upstream and Downstream TMC Link Speed - Jan & Feb',
    xaxis1 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    xaxis2 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    yaxis_nticks=13,
    yaxis=dict(title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    )

fig.show()

In [ ]:
#2018 March & April

z1 = df2018_sub_Mar.iloc[:,1:]
x1 = df2018_sub_Mar.iloc[:,0]
y1 = df2018_sub_Mar.columns[1:]

z2 = df2018_sub_Apr.iloc[:,1:]
x2 = df2018_sub_Apr.iloc[:,0]
y2 = df2018_sub_Apr.columns[1:]

trace1 = go.Heatmap(z=z1, x=y1, y=x1, colorscale='Viridis')
trace2 = go.Heatmap(z=z2, x=y2, y=x2, colorbar=dict(x=0.43), colorscale='Viridis')

fig = tools.make_subplots(rows=1, cols=2, horizontal_spacing = 0.17)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(title='Upstream and Downstream TMC Link Speed - Mar & Apr',
    xaxis1 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    xaxis2 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    yaxis_nticks=13,
    yaxis=dict(title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    )

fig.show()

In [ ]:
#2018 May & June

z1 = df2018_sub_May.iloc[:,1:]
x1 = df2018_sub_May.iloc[:,0]
y1 = df2018_sub_May.columns[1:]

z2 = df2018_sub_Jun.iloc[:,1:]
x2 = df2018_sub_Jun.iloc[:,0]
y2 = df2018_sub_Jun.columns[1:]

trace1 = go.Heatmap(z=z1, x=y1, y=x1, colorscale='Viridis')
trace2 = go.Heatmap(z=z2, x=y2, y=x2, colorbar=dict(x=0.43), colorscale='Viridis')

fig = tools.make_subplots(rows=1, cols=2, horizontal_spacing = 0.17)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(title='Upstream and Downstream TMC Link Speed - May & Jun',
    xaxis1 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    xaxis2 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    yaxis_nticks=13,
    yaxis=dict(title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    )

fig.show()

In [ ]:
#2018 Jul & August

z1 = df2018_sub_Jul.iloc[:,1:]
x1 = df2018_sub_Jul.iloc[:,0]
y1 = df2018_sub_Jul.columns[1:]

z2 = df2018_sub_Aug.iloc[:,1:]
x2 = df2018_sub_Aug.iloc[:,0]
y2 = df2018_sub_Aug.columns[1:]

trace1 = go.Heatmap(z=z1, x=y1, y=x1, colorscale='Viridis')
trace2 = go.Heatmap(z=z2, x=y2, y=x2, colorbar=dict(x=0.43), colorscale='Viridis')

fig = tools.make_subplots(rows=1, cols=2, horizontal_spacing = 0.17)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(title='Upstream and Downstream TMC Link Speed - Jul & Aug',
    xaxis1 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    xaxis2 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    yaxis_nticks=13,
    yaxis=dict(title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    )

fig.show()

In [ ]:
#2018 September & October

z1 = df2018_sub_Sep.iloc[:,1:]
x1 = df2018_sub_Sep.iloc[:,0]
y1 = df2018_sub_Sep.columns[1:]

z2 = df2018_sub_Oct.iloc[:,1:]
x2 = df2018_sub_Oct.iloc[:,0]
y2 = df2018_sub_Oct.columns[1:]

trace1 = go.Heatmap(z=z1, x=y1, y=x1, colorscale='Viridis')
trace2 = go.Heatmap(z=z2, x=y2, y=x2, colorbar=dict(x=0.43), colorscale='Viridis')

fig = tools.make_subplots(rows=1, cols=2, horizontal_spacing = 0.17)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(title='Upstream and Downstream TMC Link Speed - Sep & Oct',
    xaxis1 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    xaxis2 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    yaxis_nticks=13,
    yaxis=dict(title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    )

fig.show()

In [ ]:
#2018 Novtember & Decober

z1 = df2018_sub_Nov.iloc[:,1:]
x1 = df2018_sub_Nov.iloc[:,0]
y1 = df2018_sub_Nov.columns[1:]

z2 = df2018_sub_Dec.iloc[:,1:]
x2 = df2018_sub_Dec.iloc[:,0]
y2 = df2018_sub_Dec.columns[1:]

trace1 = go.Heatmap(z=z1, x=y1, y=x1, colorscale='Viridis')
trace2 = go.Heatmap(z=z2, x=y2, y=x2, colorbar=dict(x=0.43), colorscale='Viridis')

fig = tools.make_subplots(rows=1, cols=2, horizontal_spacing = 0.17)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(title='Upstream and Downstream TMC Link Speed - Nov & Dec',
    xaxis1 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    xaxis2 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    yaxis_nticks=13,
    yaxis=dict(title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    )

fig.show()

In [ ]:
#2018 January & February Crash Data

z1 = df_pre15_2018_Jan.iloc[:,1:]
x1 = df_pre15_2018_Jan.iloc[:,0]
y1 = df_pre15_2018_Jan.columns[1:]

z2 = df_pre15_2018_Feb.iloc[:,1:]
x2 = df_pre15_2018_Feb.iloc[:,0]
y2 = df_pre15_2018_Feb.columns[1:]

trace1 = go.Heatmap(z=z1, x=y1, y=x1, colorscale='Viridis')
trace2 = go.Heatmap(z=z2, x=y2, y=x2, colorbar=dict(x=0.43), colorscale='Viridis')

fig = tools.make_subplots(rows=1, cols=2, horizontal_spacing = 0.17)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(title='Upstream and Downstream TMC Link Speed',
    xaxis1 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    xaxis2 = dict(title_text = 'TMC LINKS', titlefont=dict(size = 20)),
    yaxis_nticks=13,
    yaxis=dict(title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    )

fig.show()

In [ ]:
crash_dn1w = df[df['LOC_DIR']=='W']['dn1_diff']
crash_dn2w = df[df['LOC_DIR']=='W']['dn2_diff']
crash_dn3w = df[df['LOC_DIR']=='W']['dn3_diff']
crash_up1w = df[df['LOC_DIR']=='W']['up1_diff']
crash_up2w = df[df['LOC_DIR']=='W']['up2_diff']
crash_up3w = df[df['LOC_DIR']=='W']['up3_diff']


trace_dn1w = go.Histogram(x=crash_dn1w, name = 'Spd diff w/ dn1')
trace_dn2w = go.Histogram(x=crash_dn2w, name = 'Spd diff w/ dn2')
trace_dn3w = go.Histogram(x=crash_dn3w, name = 'Spd diff w/ dn3')
trace_up1w = go.Histogram(x=crash_up1w, name = 'Spd diff w/ up1')
trace_up2w = go.Histogram(x=crash_up2w, name = 'Spd diff w/ up2')
trace_up3w = go.Histogram(x=crash_up3w, name = 'Spd diff w/ up3')

fig = tools.make_subplots(rows=3, cols=2)
fig.append_trace(trace_dn1w, 1, 1)
fig.append_trace(trace_dn2w, 1, 2)
fig.append_trace(trace_dn3w, 2, 1)
fig.append_trace(trace_up1w, 2, 2)
fig.append_trace(trace_up2w, 3, 1)
fig.append_trace(trace_up3w, 3, 2)

fig['layout'].update(title='Speed differences among TMCs (WB)', bargap = 0.3, 
#                      xaxis1 = dict(title="Speed (mph)"), 
#                      xaxis2 = dict(title="Speed (mph)"),
#                      xaxis3 = dict(title="Speed (mph)"),
#                      xaxis4 = dict(title="Speed (mph)"),
                     xaxis5 = dict(title="Speed (mph)"),
                     xaxis6 = dict(title="Speed (mph)"), barmode='group')
iplot(fig)

## Post 5 minutes

In [ ]:
df_post5.rename(columns ={'post_5min':'timestamp','post5_dn3_spd':'DN LINK 3', 'post5_dn2_spd':'DN LINK 2', 'post5_dn1_spd':'DN LINK 1',
                        'post5_spd':'CRASH LINK', 'post5_up1_spd':'UP LINK 1', 'post5_up2_spd':'UP LINK2', 'post5_up3_spd':'UP LINK 3'}, inplace = True)

In [ ]:
df_post5['timestamp'] = pd.to_datetime(df_post5['timestamp'])

In [ ]:
df_post5_sub2018.head()

In [ ]:
df_post5_sub2018 = df_post5_sub2018.sort_values(by='timestamp')

In [ ]:
df_post5_sub2018 = df_post5[df_post5['timestamp'].dt.year == 2018]

In [ ]:
df_post5_sub2018_Jan = df_post5[df_post5_sub2018_Jan['timestamp'].dt.month == 1]

In [ ]:
date = df_post5_sub2018['timestamp'].astype(str)
z = df_post5_sub2018.iloc[:,1:]

fig = go.Figure(data=go.Heatmap(
        z=z,      
        x=df_post5_sub2018.columns[1:],
        y=date,    
        colorscale='Viridis'))

fig.update_layout(
    title='Upstream and Downstream TMC Link Speed',
    
    xaxis = dict(
        title_text = 'TMC LINKS',
        titlefont=dict(size = 20)
    ),
    yaxis_nticks=13,
    yaxis=dict(
        title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    
    
    
    )

fig.show()

## Post 10 minutes

In [ ]:
df_post10 = df[['post_10min', 'post10_dn3_spd', 'post10_dn2_spd', 'post10_dn1_spd', 'post10_spd', \
             'post10_up1_spd', 'post10_up2_spd', 'post10_up3_spd']]

In [ ]:
df_post10.rename(columns ={'post_10min':'timestamp','post5_dn3_spd':'DN LINK 3', 'post5_dn2_spd':'DN LINK 2', 'post5_dn1_spd':'DN LINK 1',
                        'post5_spd':'CRASH LINK', 'post5_up1_spd':'UP LINK 1', 'post5_up2_spd':'UP LINK2', 'post5_up3_spd':'UP LINK 3'}, inplace = True)

In [ ]:
df_post10['timestamp'] = pd.to_datetime(df_post10['timestamp'])

In [ ]:
df_post10_sub2018 = df_post10[df_post10['timestamp'].dt.year == 2018]

In [ ]:
df_post10_sub2018 = df_post10_sub2018.sort_values(by='timestamp')

In [ ]:
date = df_post10_sub2018['timestamp'].astype(str)
z = df_post10_sub2018.iloc[:,1:]

fig = go.Figure(data=go.Heatmap(
        z=z,      
        x=df_post10_sub2018.columns[1:],
        y=date,    
        colorscale='Viridis'))

fig.update_layout(
    title='Upstream and Downstream TMC Link Speed',
    
    xaxis = dict(
        title_text = 'TMC LINKS',
        titlefont=dict(size = 20)
    ),
    yaxis_nticks=13,
    yaxis=dict(
        title_text="DATE",
#         ticktext=["Very long label", "long label", "3", "label"],
#         tickvals=[1, 2, 3, 4],
#         tickmode="array",
        titlefont=dict(size=20))
    
    
    
    )

fig.show()

In [ ]:
df=pd.DataFrame(np.random.rand(10, 4), columns=['a', 'b', 'c', 'd'])
x=[1,2,3,4,5]
y=[1,4,9,16,25]

fig, axes = plt.subplots(figsize=(8,8),nrows=2, ncols=2)
ax1=plt.subplot(2,2,1)
plt.plot(x,y)

#ax2=plt.subplot(2,2,2)
df["b"].plot(ax=axes[0,1], kind='bar', grid=True)
df["c"].plot(ax=axes[1,0], kind='bar', grid=True)
df["d"].plot(ax=axes[1,1], kind='bar', grid=True)

ax1.grid(True)
ax1.set_ylabel('Test')
ax1.set_xlabel('Test2')

#ax2.set_ylabel('Test')

In [ ]:
import plotly.express as px
df4= px.data.iris()
fig = px.scatter(df4, x="sepal_width", y="sepal_length", color="sepal_length",
                 color_continuous_scale=[(0, "red"), (0.5, "orange"), (1, "green")])

fig.show()

In [ ]:
import plotly.express as px
px.data.iris()

In [ ]:
import plotly.express as px
import numpy as np
df1 = px.data.gapminder()
fig = px.scatter_3d(df1, x='year', y='continent', z='pop', size='gdpPercap', color='lifeExp',
                    hover_data=['country'])
fig.update_layout(scene_zaxis_type="log")
fig.show()

In [ ]:
df1

In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Get Data: this ex will only use part of it (i.e. rows 750-1500)
df2 = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

start, end = 750, 1500

fig = go.Figure(data=go.Scatter3d(
    x=df2['year'][start:end],
    y=df2['continent'][start:end],
    z=df2['pop'][start:end],
    text=df['country'][start:end],
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref=750,
        size=df2['gdpPercap'][start:end],
        color = df2['lifeExp'][start:end],
        colorscale = 'Viridis',
        colorbar_title = 'Life<br>Expectancy',
        line_color='rgb(140, 140, 170)'
    )
))


fig.update_layout(height=800, width=800,
                  title='Examining Population and Life Expectancy Over Time')

fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

rs = np.random.RandomState()
rs.seed(0)

def brownian_motion(T = 1, N = 100, mu = 0.1, sigma = 0.01, S0 = 20):
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = rs.standard_normal(size = N)
    W = np.cumsum(W)*np.sqrt(dt) # standard brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*W
    S = S0*np.exp(X) # geometric brownian motion
    return S

dates = pd.date_range('2012-01-01', '2013-02-22')
T = (dates.max()-dates.min()).days / 365
N = dates.size
start_price = 100
y = brownian_motion(T, N, sigma=0.1, S0=start_price)
z = brownian_motion(T, N, sigma=0.1, S0=start_price)

fig = go.Figure(data=go.Scatter3d(
    x=dates, y=y, z=z,
    marker=dict(
        size=4,
        color=z,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))

fig.update_layout(
    width=800,
    height=700,
    autosize=False,
    scene=dict(
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=1
            ),
            eye=dict(
                x=0,
                y=1.0707,
                z=1,
            )
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    ),
)

fig.show()

In [ ]:
dd = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')


In [ ]:
dd.head()

In [ ]:
dd['Date']

In [ ]:
import plotly.figure_factory as ff
import numpy as np

# x1 = df_sub['dn3_spd']
# x2 = df_sub['dn2_spd']
x3 = df_sub['dn1_spd']
x4 = df_sub['spd']
x5 = df_sub['up1_spd']
# x6 = df_sub['up2_spd']
# x7 = df_sub['up3_spd']

hist_data = [x3, x4, x5]

group_labels = ['DN TMC1', 'Crash TMC', 'UP TMC1']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=1.0)
fig.show()